# Chapter 3. Processing Raw Text

The most important source of texts is undoubtedly the Web. It’s convenient to have existing text collections to explore, such as the corpora we saw in the previous chapters. However, you probably have your own text sources in mind, and need to learn how to access them.

The goal of this chapter is to answer the following questions:

<ol>
<li>How can we write programs to access text from local files and from the Web, in order to get hold of an unlimited range of language material?</li>

<li>How can we split documents up into individual words and punctuation symbols, so we can carry out the same kinds of analysis we did with text corpora in earlier chapters?</li> 

<li>How can we write programs to produce formatted output and save it in a file?</li>
</ol>

In order to address these questions, we will be covering key concepts in NLP, including tokenization and stemming. Along the way you will consolidate your Python knowledge and learn about strings, files, and regular expressions. Since so much text on the Web is in HTML format, we will also see how to dispense with markup.

***
    NOTE
        Important: From this chapter onwards, our program samples will assume you begin your
        interactive session or your program with the following import statements:

<div class="alert alert-block alert-info">
    >>> from __future__ import division<br>
    >>> import nltk, re, pprint
</div>

***

## Accessing Text from the Web and from Disk

### Electronic Books

A small sample of texts from Project Gutenberg appears in the NLTK corpus collection. However, you may be interested in analyzing other texts from Project Gutenberg. You can browse the catalog of 25,000 free online books at http://www.gutenberg.org/catalog/, and obtain a URL to an ASCII text file. Although 90% of the texts in Project Gutenberg are in English, it includes material in over 50 other languages, including Catalan, Chinese, Dutch, Finnish, French, German, Italian, Portuguese, and Spanish (with more than 100 texts each).

Text number 2554 is an English translation of Crime and Punishment, and we can access it as follows.

<div class="alert alert-block alert-info">
>>> from urllib import urlopen<br>
>>> url = "http://www.gutenberg.org/files/2554/2554.txt"<br>
>>> raw = urlopen(url).read()<br>
>>> type(raw)<br>
<type 'str'><br>
>>> len(raw)<br>
1176831<br>
>>> raw[:75]<br>
'The Project Gutenberg EBook of Crime and Punishment, by Fyodor Dostoevsky\r\n'
</div>
    
***    
    NOTE
        The read( ) process will take a few seconds as it downloads this large book. 
        If you’re using an Internet proxy that is not correctly detected by Python, you may need to specify the proxy manually as follows:

<div class="alert alert-block alert-info">    
>>> proxies = {'http': 'http://www.someproxy.com:3128'}<br>
>>> raw = urlopen(url, proxies=proxies).read( )<br>
</div> 
    
***
    
The variable raw contains a string with 1,176,831 characters. (We can see that it is a string, using type(raw).) This is the raw content of the book, including many details we are not interested in, such as whitespace, line breaks, and blank lines. Notice the \r and \n in the opening line of the file, which is how Python displays the special carriage return and line-feed characters (the file must have been created on a Windows machine). For our language processing, we want to break up the string into words and punctuation, as we saw in Chapter 1. This step is called tokenization, and it produces our familiar structure, a list of words and punctuation.

  
<div class="alert alert-block alert-info">    
>>> tokens = nltk.word_tokenize(raw)<br>
>>> type(tokens)<br>
<type 'list'><br>
>>> len(tokens)<br>
255809<br>
>>> tokens[:10]<br>
['The', 'Project', 'Gutenberg', 'EBook', 'of', 'Crime', 'and', 'Punishment', ',', 'by']
</div>    
    
Notice that NLTK was needed for tokenization, but not for any of the earlier tasks of opening a URL and reading it into a string. If we now take the further step of creating an NLTK text from this list, we can carry out all of the other linguistic processing we saw in Chapter 1, along with the regular list operations, such as slicing:

<div class="alert alert-block alert-info">    
>>> text = nltk.Text(tokens)<br>
>>> type(text)<br>
<type 'nltk.text.Text'><br>
>>> text[1020:1060]<br>
['CHAPTER', 'I', 'On', 'an', 'exceptionally', 'hot', 'evening', 'early', 'in',
'July', 'a', 'young', 'man', 'came', 'out', 'of', 'the', 'garret', 'in',<br>
'which', 'he', 'lodged', 'in', 'S', '.', 'Place', 'and', 'walked', 'slowly',
',', 'as', 'though', 'in', 'hesitation', ',', 'towards', 'K', '.', 'bridge', '.']<br>
>>> text.collocations()<br>
Katerina Ivanovna; Pulcheria Alexandrovna; Avdotya Romanovna; Pyotr
Petrovitch; Project Gutenberg; Marfa Petrovna; Rodion Romanovitch;<br>
Sofya Semyonovna; Nikodim Fomitch; did not; Hay Market; Andrey
Semyonovitch; old woman; Literary Archive; Dmitri Prokofitch; great
deal; United States; Praskovya Pavlovna; Porfiry Petrovitch; ear rings
</div>    
    
Notice that Project Gutenberg appears as a collocation. This is because each text downloaded from Project Gutenberg contains a header with the name of the text, the author, the names of people who scanned and corrected the text, a license, and so on. Sometimes this information appears in a footer at the end of the file. We cannot reliably detect where the content begins and ends, and so have to resort to manual inspection of the file, to discover unique strings that mark the beginning and the end, before trimming raw to be just the content and nothing else:

<div class="alert alert-block alert-info">    
>>> raw.find("PART I")<br>
5303<br>
>>> raw.rfind("End of Project Gutenberg's Crime")<br>
1157681<br>
>>> raw = raw[5303:1157681] <br>
>>> raw.find("PART I") <br>
0
</div>    
    
The find() and rfind() (“reverse find”) methods help us get the right index values to use for slicing the string . We overwrite raw with this slice, so now it begins with “PART I” and goes up to (but not including) the phrase that marks the end of the content.

This was our first brush with the reality of the Web: texts found on the Web may contain unwanted material, and there may not be an automatic way to remove it. But with a small amount of extra work we can extract the material we need.

### Dealing with HTML

Much of the text on the Web is in the form of HTML documents. You can use a web browser to save a page as text to a local file, then access this as described in the later section on files. However, if you’re going to do this often, it’s easiest to get Python to do the work directly. The first step is the same as before, using urlopen. For fun we’ll pick a BBC News story called “Blondes to die out in 200 years,” an urban legend passed along by the BBC as established scientific fact:

<div class="alert alert-block alert-info">
>>> url = "http://news.bbc.co.uk/2/hi/health/2284783.stm"<br>
>>> html = urlopen(url).read()<br>
>>> html[:60]<br>
  <!doctype html public "-//W3C//DTD HTML 4.0 Transitional//EN>
</div>

You can type print html to see the HTML content in all its glory, including meta tags, an image map, JavaScript, forms, and tables.

Getting text out of HTML is a sufficiently common task that NLTK provides a helper function nltk.clean_html(), which takes an HTML string and returns raw text. We can then tokenize this to get our familiar text structure:

<div class="alert alert-block alert-info">
>>> raw = nltk.clean_html(html)<br>
>>> tokens = nltk.word_tokenize(raw)<br>
>>> tokens<br>
['BBC', 'NEWS', '|', 'Health', '|', 'Blondes', "'", 'to', 'die', 'out', ...]
</div>

This still contains unwanted material concerning site navigation and related stories. With some trial and error you can find the start and end indexes of the content and select the tokens of interest, and initialize a text as before.

<div class="alert alert-block alert-info">    
>>> tokens = tokens[96:399]<br>
>>> text = nltk.Text(tokens)<br>
>>> text.concordance('gene')<br>
 they say too few people now carry the <b>gene</b> for blondes to last beyond the next tw<br>
t blonde hair is caused by a recessive <b>gene</b> . In order for a child to have blonde<br>
to have blonde hair , it must have the <b>gene</b> on both sides of the family in the gra<br>
there is a disadvantage of having that <b>gene</b> or by chance . They don ' t disappear<br>
ondes would disappear is if having the <b>gene</b> was a disadvantage and I do not think
</div>   
    
***    
    NOTE

        For more sophisticated processing of HTML, use the Beautiful Soup package, available at
        http://www.crummy.com/software/BeautifulSoup/.
***    

### Processing Search Engine Results

The Web can be thought of as a huge corpus of unannotated text. Web search engines provide an efficient means of searching this large quantity of text for relevant linguistic examples. The main advantage of search engines is size: since you are searching such a large set of documents, you are more likely to find any linguistic pattern you are interested in. Furthermore, you can make use of very specific patterns, which would match only one or two examples on a smaller example, but which might match tens of thousands of examples when run on the Web. A second advantage of web search engines is that they are very easy to use. Thus, they provide a very convenient tool for quickly checking a theory, to see if it is reasonable. See Table 3-1 for an example.

Table 3-1. Google hits for collocations: The number of hits for collocations involving the words absolutely or definitely, followed by one of adore, love, like, or prefer. (Liberman, in LanguageLog, 2005)

![table 3-1](googColl.png)

Unfortunately, search engines have some significant shortcomings. First, the allowable range of search patterns is severely restricted. Unlike local corpora, where you write programs to search for arbitrarily complex patterns, search engines generally only allow you to search for individual words or strings of words, sometimes with wildcards. Second, search engines give inconsistent results, and can give widely different figures when used at different times or in different geographical regions. When content has been duplicated across multiple sites, search results may be boosted. Finally, the markup in the result returned by a search engine may change unpredictably, breaking any pattern-based method of locating particular content (a problem which is ameliorated by the use of search engine APIs).

***
    NOTE

        Your Turn: Search the Web for "the of" (inside quotes). 
        Based on the large count, can we conclude that the of is a frequent collocation in English?
***

### Processing RSS Feeds

The blogosphere is an important source of text, in both formal and informal registers. With the help of a third-party Python library called the Universal Feed Parser, freely downloadable from http://feedparser.org/, we can access the content of a blog, as shown here:

<div class="alert alert-block alert-info">
>>> import feedparser<br>
>>> llog = feedparser.parse("http://languagelog.ldc.upenn.edu/nll/?feed=atom")<br>
>>> llog['feed']['title']<br>
u'Language Log'<br>
>>> len(llog.entries)<br>
15<br>
>>> post = llog.entries[2]<br>
>>> post.title<br>
u"He's My BF"<br>
>>> content = post.content[0].value<br>
>>> content[:70]<br>
u'<p>Today I was chatting with three of our visiting graduate students f'<br>
>>> nltk.word_tokenize(nltk.html_clean(content))<br>
>>> nltk.word_tokenize(nltk.clean_html(llog.entries[2].content[0].value))<br>
[u'Today', u'I', u'was', u'chatting', u'with', u'three', u'of', u'our', u'visiting',<br>
u'graduate', u'students', u'from', u'the', u'PRC', u'.', u'Thinking', u'that', u'I',<br>
u'was', u'being', u'au', u'courant', u',', u'I', u'mentioned', u'the', u'expression',<br>
u'DUI4XIANG4', u'\u5c0d\u8c61', u'("', u'boy', u'/', u'girl', u'friend', u'"', ...]
</div>

Note that the resulting strings have a u prefix to indicate that they are Unicode strings (see Text Processing with Unicode). With some further work, we can write programs to create a small corpus of blog posts, and use this as the basis for our NLP work.

### Reading Local Files

In order to read a local file, we need to use Python’s built-in open() function, followed by the read() method. Supposing you have a file document.txt, you can load its contents like this:

<div class="alert alert-block alert-info">
>>> f = open('document.txt')
>>> raw = f.read()
</div> 
    
***    
NOTE
    
    Your Turn: Create a file called document.txt using a text editor, and type in a few lines of text, and save it as plain text. If you are using IDLE, select the New Window command in the File menu, typing the required text into this window, and then saving the file as document.txt inside the directory that IDLE offers in the pop-up dialogue box. Next, in the Python interpreter, open the file using f = open('document.txt'), then inspect its contents using print f.read().
***

Various things might have gone wrong when you tried this. If the interpreter couldn’t find your file, you would have seen an error like this:

<div class="alert alert-block alert-info">
>>> f = open('document.txt')
Traceback (most recent call last):
File "<pyshell#7>", line 1, in -toplevel-
f = open('document.txt')
IOError: [Errno 2] No such file or directory: 'document.txt'
</div>

To check that the file that you are trying to open is really in the right directory, use IDLE’s Open command in the File menu; this will display a list of all the files in the directory where IDLE is running. An alternative is to examine the current directory from within Python:

<div class="alert alert-block alert-info">
>>> import os
>>> os.listdir('.')
</div>

Another possible problem you might have encountered when accessing a text file is the newline conventions, which are different for different operating systems. The built-in open() function has a second parameter for controlling how the file is opened: open('document.txt', 'rU'). 'r' means to open the file for reading (the default), and 'U' stands for “Universal”, which lets us ignore the different conventions used for marking newlines.

Assuming that you can open the file, there are several methods for reading it. The read() method creates a string with the contents of the entire file:

<div class="alert alert-block alert-info">
>>> f.read()
'Time flies like an arrow.\nFruit flies like a banana.\n'
</div>

Recall that the '\n' characters are newlines; this is equivalent to pressing Enter on a keyboard and starting a new line.

We can also read a file one line at a time using a for loop:

<div class="alert alert-block alert-info">
>>> f = open('document.txt', 'rU')
>>> for line in f:
...     print line.strip()
Time flies like an arrow.
Fruit flies like a banana.
</div>

Here we use the strip() method to remove the newline character at the end of the input line.

NLTK’s corpus files can also be accessed using these methods. We simply have to use nltk.data.find() to get the filename for any corpus item. Then we can open and read it in the way we just demonstrated:

<div class="alert alert-block alert-info">
>>> path = nltk.data.find('corpora/gutenberg/melville-moby_dick.txt')
>>> raw = open(path, 'rU').read()
</div>    

### Extracting Text from PDF, MSWord, and Other Binary Formats

ASCII text and HTML text are human-readable formats. Text often comes in binary formats—such as PDF and MSWord—that can only be opened using specialized software. Third-party libraries such as pypdf and pywin32 provide access to these formats. Extracting text from multicolumn documents is particularly challenging. For one-off conversion of a few documents, it is simpler to open the document with a suitable application, then save it as text to your local drive, and access it as described below. If the document is already on the Web, you can enter its URL in Google’s search box. The search result often includes a link to an HTML version of the document, which you can save as text.

### Capturing User Input
Sometimes we want to capture the text that a user inputs when she is interacting with our program. To prompt the user to type a line of input, call the Python function raw_input(). After saving the input to a variable, we can manipulate it just as we have done for other strings.

<div class="alert alert-block alert-info">
>>> s = raw_input("Enter some text: ")<br>
Enter some text: On an exceptionally hot evening early in July<br>
>>> print "You typed", len(nltk.word_tokenize(s)), "words."<br>
You typed 8 words.
</div>

### The NLP Pipeline
Figure 3-1 summarizes what we have covered in this section, including the process of building a vocabulary that we saw in Chapter 1. (One step, normalization, will be discussed in Normalizing Text.)

![figure 3-1](nlpPipeline.png)

Figure 3-1. The processing pipeline: We open a URL and read its HTML content, remove the markup and select a slice of characters; this is then tokenized and optionally converted into an nltk.Text object; we can also lowercase all the words and extract the vocabulary.

There’s a lot going on in this pipeline. To understand it properly, it helps to be clear about the type of each variable that it mentions. We find out the type of any Python object x using type(x); e.g., type(1) is <int> since 1 is an integer.

When we load the contents of a URL or file, and when we strip out HTML markup, we are dealing with strings, Python’s <str> data type (we will learn more about strings in Strings: Text Processing at the Lowest Level):

<div class="alert alert-block alert-info">
>>> raw = open('document.txt').read()<br>
>>> type(raw)<br>
<type 'str'>
</div>
    
When we tokenize a string we produce a list (of words), and this is Python’s <list> type. Normalizing and sorting lists produces other lists:

<div class="alert alert-block alert-info">
>>> tokens = nltk.word_tokenize(raw)<br>
>>> type(tokens)<br>
<type 'list'><br>
>>> words = [w.lower() for w in tokens]<br>
>>> type(words)<br>
<type 'list'><br>
>>> vocab = sorted(set(words))<br>
>>> type(vocab)<br>
<type 'list'>
</div>
    
The type of an object determines what operations you can perform on it. So, for example, we can append to a list but not to a string:

<div class="alert alert-block alert-info">
>>> vocab.append('blog')<br>
>>> raw.append('blog')<br>
Traceback (most recent call last):<br>
  File "<stdin>", line 1, in <module><br>
AttributeError: 'str' object has no attribute 'append'
</div>    
    
Similarly, we can concatenate strings with strings, and lists with lists, but we cannot concatenate strings with lists:

<div class="alert alert-block alert-info">
>>> query = 'Who knows?'<br>
>>> beatles = ['john', 'paul', 'george', 'ringo']<br>
>>> query + beatles<br>
Traceback (most recent call last):<br>
  File "<stdin>", line 1, in <module><br>
TypeError: cannot concatenate 'str' and 'list' objects
</div>
    
In the next section, we examine strings more closely and further explore the relationship between strings and lists.

## Strings: Text Processing at the Lowest Level

It’s time to study a fundamental data type that we’ve been studiously avoiding so far. In earlier chapters we focused on a text as a list of words. We didn’t look too closely at words and how they are handled in the programming language. By using NLTK’s corpus interface we were able to ignore the files that these texts had come from. The contents of a word, and of a file, are represented by programming languages as a fundamental data type known as a string. In this section, we explore strings in detail, and show the connection between strings, words, texts, and files.

### Basic Operations with Strings
Strings are specified using single quotes 1 or double quotes 2, as shown in the following code example. If a string contains a single quote, we must backslash-escape the quote 3 so Python knows a literal quote character is intended, or else put the string in double quotes 2. Otherwise, the quote inside the string 4 will be interpreted as a close quote, and the Python interpreter will report a syntax error:

<div class="alert alert-block alert-info">
>>> monty = 'Monty Python' 1<br>
>>> monty<br>
'Monty Python'<br>
>>> circus = "Monty Python's Flying Circus" 2<br>
>>> circus<br>
"Monty Python's Flying Circus"<br>
>>> circus = 'Monty Python\'s Flying Circus' 3<br>
>>> circus<br>
"Monty Python's Flying Circus"<br>
>>> circus = 'Monty Python's Flying Circus' 4<br>
  File "<stdin>", line 1<br>
    circus = 'Monty Python's Flying Circus'<br>
                           ^<br>
SyntaxError: invalid syntax
</div>    
    
Sometimes strings go over several lines. Python provides us with various ways of entering them. In the next example, a sequence of two strings is joined into a single string. We need to use backslash 1 or parentheses 2 so that the interpreter knows that the statement is not complete after the first line.

<div class="alert alert-block alert-info">
>>> couplet = "Shall I compare thee to a Summer's day?"\<br>
...           "Thou are more lovely and more temperate:" 1<br>
>>> print couplet<br>
Shall I compare thee to a Summer's day?Thou are more lovely and more temperate:<br>
>>> couplet = ("Rough winds do shake the darling buds of May,"<br>
...           "And Summer's lease hath all too short a date:") 2<br>
>>> print couplet<br>
Rough winds do shake the darling buds of May,And Summer's lease hath all too short a date:<br>
</div>    
    
Unfortunately these methods do not give us a newline between the two lines of the sonnet. Instead, we can use a triple-quoted string as follows:

<div class="alert alert-block alert-info">
>>> couplet = """Shall I compare thee to a Summer's day?<br>
... Thou are more lovely and more temperate:"""<br>
>>> print couplet<br>
Shall I compare thee to a Summer's day?<br>
Thou are more lovely and more temperate:<br>
>>> couplet = '''Rough winds do shake the darling buds of May,<br>
... And Summer's lease hath all too short a date:'''<br>
>>> print couplet<br>
Rough winds do shake the darling buds of May,<br>
And Summer's lease hath all too short a date:<br>
</div>    
    
Now that we can define strings, we can try some simple operations on them. First let’s look at the + operation, known as concatenation 1. It produces a new string that is a copy of the two original strings pasted together end-to-end. Notice that concatenation doesn’t do anything clever like insert a space between the words. We can even multiply strings 2:

<div class="alert alert-block alert-info">
>>> 'very' + 'very' + 'very' 1<br>
'veryveryvery'<br>
>>> 'very' * 3 2<br>
'veryveryvery'<br>
</div>
    
***    
    NOTE
        Your Turn: Try running the following code, then try to use your understanding of 
        the string + and * operations to figure out how it works. Be careful to distinguish 
        between the string ' ', which is a single whitespace character, and '', which is the empty string.
***    

<div class="alert alert-block alert-info">
>>> a = [1, 2, 3, 4, 5, 6, 7, 6, 5, 4, 3, 2, 1]<br>
>>> b = [' ' * 2 * (7 - i) + 'very' * i for i in a]<br>
>>> for line in b:<br>
...     print line
</div>
    
We’ve seen that the addition and multiplication operations apply to strings, not just numbers. However, note that we cannot use subtraction or division with strings:

<div class="alert alert-block alert-info">
>>> 'very' - 'y'<br>
Traceback (most recent call last):<br>
  File "<stdin>", line 1, in <module><br>
TypeError: unsupported operand type(s) for -: 'str' and 'str'<br>
>>> 'very' / 2<br>
Traceback (most recent call last):<br>
  File "<stdin>", line 1, in <module><br>
TypeError: unsupported operand type(s) for /: 'str' and 'int'
</div>
    
These error messages are another example of Python telling us that we have got our data types in a muddle. In the first case, we are told that the operation of subtraction (i.e., -) cannot apply to objects of type str (strings), while in the second, we are told that division cannot take str and int as its two operands.

### Printing Strings
    
So far, when we have wanted to look at the contents of a variable or see the result of a calculation, we have just typed the variable name into the interpreter. We can also see the contents of a variable using the print statement:

<div class="alert alert-block alert-info">
>>> print monty<br>
Monty Python
</div>
    
Notice that there are no quotation marks this time. When we inspect a variable by typing its name in the interpreter, the interpreter prints the Python representation of its value. Since it’s a string, the result is quoted. However, when we tell the interpreter to print the contents of the variable, we don’t see quotation characters, since there are none inside the string.

The print statement allows us to display more than one item on a line in various ways, as shown here:

<div class="alert alert-block alert-info">
>>> grail = 'Holy Grail'<br>
>>> print monty + grail<br>
Monty PythonHoly Grail<br>
>>> print monty, grail<br>
Monty Python Holy Grail<br>
>>> print monty, "and the", grail<br>
Monty Python and the Holy Grail
</div>
    
### Accessing Individual Characters
    
As we saw in A Closer Look at Python: Texts as Lists of Words for lists, strings are indexed, starting from zero. When we index a string, we get one of its characters (or letters). A single character is nothing special—it’s just a string of length 1.

<div class="alert alert-block alert-info">
>>> monty[0]<br>
'M'<br>
>>> monty[3]<br>
't'<br>
>>> monty[5]<br>
' '
</div>
    
As with lists, if we try to access an index that is outside of the string, we get an error:

<div class="alert alert-block alert-info">
>>> monty[20]<br>
Traceback (most recent call last):<br>
  File "<stdin>", line 1, in ?<br>
IndexError: string index out of range
</div>    
    
Again as with lists, we can use negative indexes for strings, where -1 is the index of the last character 1. Positive and negative indexes give us two ways to refer to any position in a string. In this case, when the string had a length of 12, indexes 5 and -7 both refer to the same character (a space). (Notice that 5 = len(monty) - 7.)

<div class="alert alert-block alert-info">
>>> monty[-1] 1<br>
'n'<br>
>>> monty[5]<br>
' '<br>
>>> monty[-7]<br>
' '
</div>
    
We can write for loops to iterate over the characters in strings. This print statement ends with a trailing comma, which is how we tell Python not to print a newline at the end.

<div class="alert alert-block alert-info">
>>> sent = 'colorless green ideas sleep furiously'<br>
>>> for char in sent:<br>
...     print char,<br>
...<br>
c o l o r l e s s   g r e e n   i d e a s   s l e e p   f u r i o u s l y
</div>    
    
We can count individual characters as well. We should ignore the case distinction by normalizing everything to lowercase, and filter out non-alphabetic characters:

<div class="alert alert-block alert-info">
>>> from nltk.corpus import gutenberg<br>
>>> raw = gutenberg.raw('melville-moby_dick.txt')<br>
>>> fdist = nltk.FreqDist(ch.lower() for ch in raw if ch.isalpha())<br>
>>> fdist.keys()<br>
['e', 't', 'a', 'o', 'n', 'i', 's', 'h', 'r', 'l', 'd', 'u', 'm', 'c', 'w',<br>
'f', 'g', 'p', 'b', 'y', 'v', 'k', 'q', 'j', 'x', 'z']
</div>    

This gives us the letters of the alphabet, with the most frequently occurring letters listed first (this is quite complicated and we’ll explain it more carefully later). You might like to visualize the distribution using fdist.plot(). The relative character frequencies of a text can be used in automatically identifying the language of the text.

### Accessing Substrings
    
String slicing: The string Monty Python is shown along with its positive and negative indexes; two substrings are selected using “slice” notation. The slice [m,n] contains the characters from position m through n-1.
Figure 3-2. String slicing: The string Monty Python is shown along with its positive and negative indexes; two substrings are selected using “slice” notation. The slice [m,n] contains the characters from position m through n-1.

A substring is any continuous section of a string that we want to pull out for further processing. We can easily access substrings using the same slice notation we used for lists (see Figure 3-2). For example, the following code accesses the substring starting at index 6, up to (but not including) index 10:

<div class="alert alert-block alert-info">
>>> monty[6:10]<br>
'Pyth'
</div>
    
Here we see the characters are 'P', 'y', 't', and 'h', which correspond to monty[6] ... monty[9] but not monty[10]. This is because a slice starts at the first index but finishes one before the end index.

We can also slice with negative indexes—the same basic rule of starting from the start index and stopping one before the end index applies; here we stop before the space character.

<div class="alert alert-block alert-info">
>>> monty[-12:-7]<br>
'Monty'
</div>    
    
As with list slices, if we omit the first value, the substring begins at the start of the string. If we omit the second value, the substring continues to the end of the string:

<div class="alert alert-block alert-info">
>>> monty[:5]<br>
'Monty'<br>
>>> monty[6:]<br>
'Python'
</div>
    
We test if a string contains a particular substring using the in operator, as follows:

<div class="alert alert-block alert-info">
>>> phrase = 'And now for something completely different'<br>
>>> if 'thing' in phrase:<br>
...     print 'found "thing"'<br>
found "thing"
</div>    
    
We can also find the position of a substring within a string, using find():

<div class="alert alert-block alert-info">
>>> monty.find('Python')<br>
6
</div>    
    
***    
    NOTE
        Your Turn: Make up a sentence and assign it to a variable, e.g., sent = 'my sentence...'. 
        Now write slice expressions to pull out individual words. 
        (This is obviously not a convenient way to process the words of a text!)
***
    
### More Operations on Strings
    
Python has comprehensive support for processing strings. A summary, including some operations we haven’t seen yet, is shown in Table 3-2. For more information on strings, type help(str) at the Python prompt.

Table 3-2. Useful string methods: Operations on strings in addition to the string tests shown in Table 1-4; all methods produce a new string or list

![table 3-2](regexTab1.png)
![table 3-2](regexTab2.png)    

### The Difference Between Lists and Strings
    
Strings and lists are both kinds of sequence. We can pull them apart by indexing and slicing them, and we can join them together by concatenating them. However, we cannot join strings and lists:

<div class="alert alert-block alert-info">
>>> query = 'Who knows?'<br>
>>> beatles = ['John', 'Paul', 'George', 'Ringo']<br>
>>> query[2]<br>
'o'<br>
>>> beatles[2]<br>
'George'<br>
>>> query[:2]<br>
'Wh'<br>
>>> beatles[:2]<br>
['John', 'Paul']<br>
>>> query + " I don't"<br>
"Who knows? I don't"<br>
>>> beatles + 'Brian'<br>
Traceback (most recent call last):<br>
  File "<stdin>", line 1, in <module><br>
TypeError: can only concatenate list (not "str") to list<br>
>>> beatles + ['Brian']<br>
['John', 'Paul', 'George', 'Ringo', 'Brian']
</div>
    
When we open a file for reading into a Python program, we get a string corresponding to the contents of the whole file. If we use a for loop to process the elements of this string, all we can pick out are the individual characters—we don’t get to choose the granularity. By contrast, the elements of a list can be as big or small as we like: for example, they could be paragraphs, sentences, phrases, words, characters. So lists have the advantage that we can be flexible about the elements they contain, and correspondingly flexible about any downstream processing. Consequently, one of the first things we are likely to do in a piece of NLP code is tokenize a string into a list of strings (Regular Expressions for Tokenizing Text). Conversely, when we want to write our results to a file, or to a terminal, we will usually format them as a string (Formatting: From Lists to Strings).

Lists and strings do not have exactly the same functionality. Lists have the added power that you can change their elements:

<div class="alert alert-block alert-info">
>>> beatles[0] = "John Lennon"<br>
>>> del beatles[-1]<br>
>>> beatles<br>
['John Lennon', 'Paul', 'George']
</div>
    
On the other hand, if we try to do that with a string—changing the 0th character in query to 'F'—we get:

<div class="alert alert-block alert-info">
>>> query[0] = 'F'<br>
Traceback (most recent call last):<br>
  File "<stdin>", line 1, in ?<br>
TypeError: object does not support item assignment
</div>
    
This is because strings are immutable: you can’t change a string once you have created it. However, lists are mutable, and their contents can be modified at any time. As a result, lists support operations that modify the original value rather than producing a new value.

***    
    NOTE
        Your Turn: Consolidate your knowledge of strings by trying some of the exercises on strings at the end of this chapter.
***

## Text Processing with Unicode
Our programs will often need to deal with different languages, and different character sets. The concept of “plain text” is a fiction. If you live in the English-speaking world you probably use ASCII, possibly without realizing it. If you live in Europe you might use one of the extended Latin character sets, containing such characters as “ø” for Danish and Norwegian, “ő” for Hungarian, “ñ” for Spanish and Breton, and “ň” for Czech and Slovak. In this section, we will give an overview of how to use Unicode for processing texts that use non-ASCII character sets.

### What Is Unicode?
Unicode supports over a million characters. Each character is assigned a number, called a code point. In Python, code points are written in the form \uXXXX, where XXXX is the number in four-digit hexadecimal form.

Within a program, we can manipulate Unicode strings just like normal strings. However, when Unicode characters are stored in files or displayed on a terminal, they must be encoded as a stream of bytes. Some encodings (such as ASCII and Latin-2) use a single byte per code point, so they can support only a small subset of Unicode, enough for a single language. Other encodings (such as UTF-8) use multiple bytes and can represent the full range of Unicode characters.

Text in files will be in a particular encoding, so we need some mechanism for translating it into Unicode—translation into Unicode is called decoding. Conversely, to write out Unicode to a file or a terminal, we first need to translate it into a suitable encoding—this translation out of Unicode is called encoding, and is illustrated in Figure 3-3.

![figure 3-3](unicode.png)
<i>Figure 3-3. Unicode decoding and encoding.</i>

From a Unicode perspective, characters are abstract entities that can be realized as one or more glyphs. Only glyphs can appear on a screen or be printed on paper. A font is a mapping from characters to glyphs.

### Extracting Encoded Text from Files
Let’s assume that we have a small text file, and that we know how it is encoded. For example, polish-lat2.txt, as the name suggests, is a snippet of Polish text (from the Polish Wikipedia; see http://pl.wikipedia.org/wiki/Biblioteka_Pruska). This file is encoded as Latin-2, also known as ISO-8859-2. The function nltk.data.find() locates the file for us.

<div class="alert alert-block alert-info">
>>> path = nltk.data.find('corpora/unicode_samples/polish-lat2.txt')
</div>
The Python codecs module provides functions to read encoded data into Unicode strings, and to write out Unicode strings in encoded form. The codecs.open() function takes an encoding parameter to specify the encoding of the file being read or written. So let’s import the codecs module, and call it with the encoding 'latin2' to open our Polish file as Unicode:

<div class="alert alert-block alert-info">
>>> import codecs<br>
>>> f = codecs.open(path, encoding='latin2')
</div>
    
For a list of encoding parameters allowed by codecs, see http://docs.python.org/lib/standard-encodings.html. Note that we can write Unicode-encoded data to a file using f = codecs.open(path, 'w', encoding='utf-8').

Text read from the file object f will be returned in Unicode. As we pointed out earlier, in order to view this text on a terminal, we need to encode it, using a suitable encoding. The Python-specific encoding unicode_escape is a dummy encoding that converts all non-ASCII characters into their \uXXXX representations. Code points above the ASCII 0–127 range but below 256 are represented in the two-digit form \xXX.

<div class="alert alert-block alert-info">
>>> for line in f:<br>
...     line = line.strip()<br>
...     print line.encode('unicode_escape')<br>
Pruska Biblioteka Pa\u0144stwowa. Jej dawne zbiory znane pod nazw\u0105<br>
"Berlinka" to skarb kultury i sztuki niemieckiej. Przewiezione przez<br>
Niemc\xf3w pod koniec II wojny \u015bwiatowej na Dolny \u015al\u0105sk, zosta\u0142y<br>
odnalezione po 1945 r. na terytorium Polski. Trafi\u0142y do Biblioteki<br>
Jagiello\u0144skiej w Krakowie, obejmuj\u0105 ponad 500 tys. zabytkowych<br>
archiwali\xf3w, m.in. manuskrypty Goethego, Mozarta, Beethovena, Bacha.
</div>
    
The first line in this output illustrates a Unicode escape string preceded by the \u escape string, namely \u0144. The relevant Unicode character will be displayed on the screen as the glyph ń. In the third line of the preceding example, we see \xf3, which corresponds to the glyph ó, and is within the 128–255 range.

In Python, a Unicode string literal can be specified by preceding an ordinary string literal with a u, as in u'hello'. Arbitrary Unicode characters are defined using the \uXXXX escape sequence inside a Unicode string literal. We find the integer ordinal of a character using ord(). For example:

<div class="alert alert-block alert-info">
>>> ord('a')<br>
97
</div>
    
The hexadecimal four-digit notation for 97 is 0061, so we can define a Unicode string literal with the appropriate escape sequence:

<div class="alert alert-block alert-info">
>>> a = u'\u0061'<br>
>>> a<br>
u'a'<br>
>>> print a<br>
a
</div>
    
Notice that the Python print statement is assuming a default encoding of the Unicode character, namely ASCII. However, ń is outside the ASCII range, so cannot be printed unless we specify an encoding. In the following example, we have specified that print should use the repr() of the string, which outputs the UTF-8 escape sequences (of the form \xXX) rather than trying to render the glyphs.

<div class="alert alert-block alert-info">
>>> nacute = u'\u0144'<br>
>>> nacute<br>
u'\u0144'<br>
>>> nacute_utf = nacute.encode('utf8')<br>
>>> print repr(nacute_utf)<br>
'\xc5\x84'
</div>
    
If your operating system and locale are set up to render UTF-8 encoded characters, you ought to be able to give the Python command print nacute_utf and see ń on your screen.

***
    NOTE
        There are many factors determining what glyphs are rendered on your screen. 
        If you are sure that you have the correct encoding, but your Python code is still 
        failing to produce the glyphs you expected, you should also check that you 
        have the necessary fonts installed on your system.
***
    
The module unicodedata lets us inspect the properties of Unicode characters. In the following example, we select all characters in the third line of our Polish text outside the ASCII range and print their UTF-8 escaped value, followed by their code point integer using the standard Unicode convention (i.e., prefixing the hex digits with U+), followed by their Unicode name.

<div class="alert alert-block alert-info">
>>> import unicodedata<br>
>>> lines = codecs.open(path, encoding='latin2').readlines()<br>
>>> line = lines[2]<br>
>>> print line.encode('unicode_escape')<br>
Niemc\xf3w pod koniec II wojny \u015bwiatowej na Dolny \u015al\u0105sk, zosta\u0142y\n<br>
>>> for c in line:<br>
...     if ord(c) > 127:<br>
...         print '%r U+%04x %s' % (c.encode('utf8'), ord(c), unicodedata.name(c))<br>
'\xc3\xb3' U+00f3 LATIN SMALL LETTER O WITH ACUTE<br>
'\xc5\x9b' U+015b LATIN SMALL LETTER S WITH ACUTE<br>
'\xc5\x9a' U+015a LATIN CAPITAL LETTER S WITH ACUTE<br>
'\xc4\x85' U+0105 LATIN SMALL LETTER A WITH OGONEK<br>
'\xc5\x82' U+0142 LATIN SMALL LETTER L WITH STROKE
</div>    

If you replace the %r (which yields the repr() value) by %s in the format string of the preceding code sample, and if your system supports UTF-8, you should see an output like the following:

<div class="alert alert-block alert-info">
ó U+00f3 LATIN SMALL LETTER O WITH ACUTE<br>
ś U+015b LATIN SMALL LETTER S WITH ACUTE<br>
Ś U+015a LATIN CAPITAL LETTER S WITH ACUTE<br>
ą U+0105 LATIN SMALL LETTER A WITH OGONEK<br>
ł U+0142 LATIN SMALL LETTER L WITH STROKE
</div>
    
Alternatively, you may need to replace the encoding 'utf8' in the example by 'latin2', again depending on the details of your system.

The next examples illustrate how Python string methods and the re module accept Unicode strings.

<div class="alert alert-block alert-info">
>>> line.find(u'zosta\u0142y')<br>
54<br>
>>> line = line.lower()<br>
>>> print line.encode('unicode_escape')<br>
niemc\xf3w pod koniec ii wojny \u015bwiatowej na dolny \u015bl\u0105sk, zosta\u0142y\n<br>
>>> import re<br>
>>> m = re.search(u'\u015b\w*', line)<br>
>>> m.group()<br>
u'\u015bwiatowej'
</div> 
    
NLTK tokenizers allow Unicode strings as input, and correspondingly yield Unicode strings as output.

<div class="alert alert-block alert-info">
>>> nltk.word_tokenize(line)  <br>
[u'niemc\xf3w', u'pod', u'koniec', u'ii', u'wojny', u'\u015bwiatowej',<br>
u'na', u'dolny', u'\u015bl\u0105sk', u'zosta\u0142y']
</div>
    
### Using Your Local Encoding in Python
If you are used to working with characters in a particular local encoding, you probably want to be able to use your standard methods for inputting and editing strings in a Python file. In order to do this, you need to include the string '# -*- coding: <coding> -*-' as the first or second line of your file. Note that <coding> has to be a string like 'latin-1', 'big5', or 'utf-8' (see Figure 3-4).

![figure 1-1](encoded.png)

<i>Figure 3-4. Unicode and IDLE: UTF-8 encoded string literals in the IDLE editor; this requires that an appropriate font is set in IDLE’s preferences; here we have chosen Courier CE.</i>
    
Figure 3-4 also illustrates how regular expressions can use encoded strings.

## Regular Expressions for Detecting Word Patterns
Many linguistic processing tasks involve pattern matching. For example, we can find words ending with ed using endswith('ed'). We saw a variety of such “word tests” in Table 1-4. Regular expressions give us a more powerful and flexible method for describing the character patterns we are interested in.

***
    NOTE
        There are many other published introductions to regular expressions, organized
        around the syntax of regular expressions and applied to searching text files. 
        Instead of doing this again, we focus on the use of regular expressions at different
        stages of linguistic processing. As usual, we’ll adopt a problem-based approach and
        present new features only as they are needed to solve practical problems. 
        In our discussion we will mark regular expressions using chevrons like this: «patt».
***

To use regular expressions in Python, we need to import the re library using: import re. We also need a list of words to search; we’ll use the Words Corpus again (Lexical Resources). We will preprocess it to remove any proper names.

<div class="alert alert-block alert-info">
>>> import re<br>
>>> wordlist = [w for w in nltk.corpus.words.words('en') if w.islower()]
</div>
    
### Using Basic Metacharacters
Let’s find words ending with ed using the regular expression «ed\\$». We will use the re.search(p, s) function to check whether the pattern p can be found somewhere inside the string s. We need to specify the characters of interest, and use the dollar sign, which has a special behavior in the context of regular expressions in that it matches the end of the word:

<div class="alert alert-block alert-info">
>>> [w for w in wordlist if re.search('ed\\$', w)]<br>
['abaissed', 'abandoned', 'abased', 'abashed', 'abatised', 'abed', 'aborted', ...]
</div>
    
The . wildcard symbol matches any single character. Suppose we have room in a crossword puzzle for an eight-letter word, with j as its third letter and t as its sixth letter. In place of each blank cell we use a period:

<div class="alert alert-block alert-info">
>>> [w for w in wordlist if re.search('^..j..t..\\$', w)]<br>
['abjectly', 'adjuster', 'dejected', 'dejectly', 'injector', 'majestic', ...]
</div>

***
    NOTE
        Your Turn: The caret symbol ^ matches the start of a string, just like the \\$ matches the end. 
        What results do we get with the example just shown if we leave out both of these, 
        and search for «..j..t..»?
***

Finally, the ? symbol specifies that the previous character is optional. Thus «^e-?mail\\\$» will match both email and e-mail. We could count the total number of occurrences of this word (in either spelling) in a text using sum(1 for w in text if re.search('^e-?mail\\\$', w)).

### Ranges and Closures
The T9 system is used for entering text on mobile phones (see Figure 3-5). Two or more words that are entered with the same sequence of keystrokes are known as textonyms. For example, both hole and golf are entered by pressing the sequence 4653. What other words could be produced with the same sequence? Here we use the regular expression «^[ghi][mno][jlk][def]\\$»:

<div class="alert alert-block alert-info">
>>> [w for w in wordlist if re.search('^[ghi][mno][jlk][def]\\$', w)]<br>
['gold', 'golf', 'hold', 'hole']
</div>
    
The first part of the expression, «^[ghi]», matches the start of a word followed by g, h, or i. The next part of the expression, «[mno]», constrains the second character to be m, n, or o. The third and fourth characters are also constrained. Only four words satisfy all these constraints. Note that the order of characters inside the square brackets is not significant, so we could have written «^[hig][nom][ljk][fed]\\$» and matched the same words.

T9: Text on 9 keys.
![figure 1-1](t9.png)
<i>Figure 3-5. T9: Text on 9 keys.</i>

***
    NOTE
        Your Turn: Look for some “finger-twisters,” by searching for words that use only
        part of the number-pad. For example «^[ghijklmno]+\\$», or more concisely, «^[g-o]+\\$», 
        will match words that only use keys 4, 5, 6 in the center row, and «^[a-fj-o]+\\$» 
        will match words that use keys 2, 3, 5, 6 in the top-right corner. What do - and + mean?
***

Let’s explore the + symbol a bit further. Notice that it can be applied to individual letters, or to bracketed sets of letters:

<div class="alert alert-block alert-info">
>>> chat_words = sorted(set(w for w in nltk.corpus.nps_chat.words()))<br>
>>> [w for w in chat_words if re.search('^m+i+n+e+\\$', w)]<br>
['miiiiiiiiiiiiinnnnnnnnnnneeeeeeeeee', 'miiiiiinnnnnnnnnneeeeeeee', 'mine',<br>
'mmmmmmmmiiiiiiiiinnnnnnnnneeeeeeee']
>>> [w for w in chat_words if re.search('^[ha]+\\$', w)]<br>
['a', 'aaaaaaaaaaaaaaaaa', 'aaahhhh', 'ah', 'ahah', 'ahahah', 'ahh',<br>
'ahhahahaha', 'ahhh', 'ahhhh', 'ahhhhhh', 'ahhhhhhhhhhhhhh', 'h', 'ha', 'haaa',<br>
'hah', 'haha', 'hahaaa', 'hahah', 'hahaha', 'hahahaa', 'hahahah', 'hahahaha', ...]
</div>    
    
It should be clear that + simply means “one or more instances of the preceding item,” which could be an individual character like m, a set like [fed], or a range like [d-f]. Now let’s replace + with *, which means “zero or more instances of the preceding item.” The regular expression «^m*i*n*e*\\$» will match everything that we found using «^m+i+n+e+\\$», but also words where some of the letters don’t appear at all, e.g., me, min, and mmmmm. Note that the + and * symbols are sometimes referred to as Kleene closures, or simply closures.

The ^ operator has another function when it appears as the first character inside square brackets. For example, «[^aeiouAEIOU]» matches any character other than a vowel. We can search the NPS Chat Corpus for words that are made up entirely of non-vowel characters using «^[^aeiouAEIOU]+\\$» to find items like these: :):):), grrr, cyb3r, and zzzzzzzz. Notice this includes non-alphabetic characters.

Here are some more examples of regular expressions being used to find tokens that match a particular pattern, illustrating the use of some new symbols: \, {}, (), and |.

<div class="alert alert-block alert-info">
>>> wsj = sorted(set(nltk.corpus.treebank.words()))<br>
>>> [w for w in wsj if re.search('^[0-9]+\.[0-9]+\\$', w)]<br>
['0.0085', '0.05', '0.1', '0.16', '0.2', '0.25', '0.28', '0.3', '0.4', '0.5',<br>
'0.50', '0.54', '0.56', '0.60', '0.7', '0.82', '0.84', '0.9', '0.95', '0.99',<br>
'1.01', '1.1', '1.125', '1.14', '1.1650', '1.17', '1.18', '1.19', '1.2', ...]<br>
>>> [w for w in wsj if re.search('^[A-Z]+\\\$\\$', w)]<br>
['C\\$', 'US\\$']<br>
>>> [w for w in wsj if re.search('^[0-9]{4}\\$', w)]<br>
['1614', '1637', '1787', '1901', '1903', '1917', '1925', '1929', '1933', ...]<br>
>>> [w for w in wsj if re.search('^[0-9]+-[a-z]{3,5}\\$', w)]<br>
['10-day', '10-lap', '10-year', '100-share', '12-point', '12-year', ...]<br>
>>> [w for w in wsj if re.search('^[a-z]{5,}-[a-z]{2,3}-[a-z]{,6}\\$', w)]<br>
['black-and-white', 'bread-and-butter', 'father-in-law', 'machine-gun-toting',<br>
'savings-and-loan']<br>
>>> [w for w in wsj if re.search('(ed|ing)\\$', w)]<br>
['62%-owned', 'Absorbed', 'According', 'Adopting', 'Advanced', 'Advancing', ...]<br>
</div>

***
    NOTE
        Your Turn: Study the previous examples and try to work out 
        what the \, {}, (), and | notations mean before you read on.
***

You probably worked out that a backslash means that the following character is deprived of its special powers and must literally match a specific character in the word. Thus, while . is special, \. only matches a period. The braced expressions, like {3,5}, specify the number of repeats of the previous item. The pipe character indicates a choice between the material on its left or its right. Parentheses indicate the scope of an operator, and they can be used together with the pipe (or disjunction) symbol like this: «w(i|e|ai|oo)t», matching wit, wet, wait, and woot. It is instructive to see what happens when you omit the parentheses from the last expression in the example, and search for «ed|ing\\$».

The metacharacters we have seen are summarized in Table 3-3.

![table 3-3](regexOp1.png)
![table 3-3](regexOp2.png)
Table 3-3. Basic regular expression metacharacters, including wildcards, ranges, and closures

To the Python interpreter, a regular expression is just like any other string. If the string contains a backslash followed by particular characters, it will interpret these specially. For example, \b would be interpreted as the backspace character. In general, when using regular expressions containing backslash, we should instruct the interpreter not to look inside the string at all, but simply to pass it directly to the re library for processing. We do this by prefixing the string with the letter r, to indicate that it is a raw string. For example, the raw string r'\band\b' contains two \b symbols that are interpreted by the re library as matching word boundaries instead of backspace characters. If you get into the habit of using r'...' for regular expressions—as we will do from now on—you will avoid having to think about these complications.

## Useful Applications of Regular Expressions
The previous examples all involved searching for words w that match some regular expression regexp using re.search(regexp, w). Apart from checking whether a regular expression matches a word, we can use regular expressions to extract material from words, or to modify words in specific ways.

### Extracting Word Pieces
The re.findall() (“find all”) method finds all (non-overlapping) matches of the given regular expression. Let’s find all the vowels in a word, then count them:

<div class="alert alert-block alert-info">
>>> word = 'supercalifragilisticexpialidocious'<br>
>>> re.findall(r'[aeiou]', word)<br>
['u', 'e', 'a', 'i', 'a', 'i', 'i', 'i', 'e', 'i', 'a', 'i', 'o', 'i', 'o', 'u']<br>
>>> len(re.findall(r'[aeiou]', word))<br>
16
</div>    
    
Let’s look for all sequences of two or more vowels in some text, and determine their relative frequency:

<div class="alert alert-block alert-info">
>>> wsj = sorted(set(nltk.corpus.treebank.words()))<br>
>>> fd = nltk.FreqDist(vs for word in wsj<br>
...                       for vs in re.findall(r'[aeiou]{2,}', word))<br>
>>> fd.items()<br>
[('io', 549), ('ea', 476), ('ie', 331), ('ou', 329), ('ai', 261), ('ia', 253),<br>
('ee', 217), ('oo', 174), ('ua', 109), ('au', 106), ('ue', 105), ('ui', 95),<br>
('ei', 86), ('oi', 65), ('oa', 59), ('eo', 39), ('iou', 27), ('eu', 18), ...]
</div>
    
***
    NOTE
        Your Turn: In the W3C Date Time Format, dates are represented like this: 2009-12-31. 
        Replace the ? in the following Python code with a regular expression, 
        in order to convert the string '2009-12-31' to a list of integers [2009, 12, 31]:

        [int(n) for n in re.findall(?, '2009-12-31')]
***
    
### Doing More with Word Pieces
Once we can use re.findall() to extract material from words, there are interesting things to do with the pieces, such as glue them back together or plot them.

It is sometimes noted that English text is highly redundant, and it is still easy to read when word-internal vowels are left out. For example, declaration becomes dclrtn, and inalienable becomes inlnble, retaining any initial or final vowel sequences. The regular expression in our next example matches initial vowel sequences, final vowel sequences, and all consonants; everything else is ignored. This three-way disjunction is processed left-to-right, and if one of the three parts matches the word, any later parts of the regular expression are ignored. We use re.findall() to extract all the matching pieces, and ''.join() to join them together (see Formatting: From Lists to Strings for more about the join operation).

<div class="alert alert-block alert-info">
>>> regexp = r'^[AEIOUaeiou]+|[AEIOUaeiou]+$|[^AEIOUaeiou]'<br>
>>> def compress(word):<br>
...     pieces = re.findall(regexp, word)<br>
...     return ''.join(pieces)<br>
...<br>
>>> english_udhr = nltk.corpus.udhr.words('English-Latin1')<br>
>>> print nltk.tokenwrap(compress(w) for w in english_udhr[:75])<br>
Unvrsl Dclrtn of Hmn Rghts Prmble Whrs rcgntn of the inhrnt dgnty and<br>
of the eql and inlnble rghts of all mmbrs of the hmn fmly is the fndtn<br>
of frdm , jstce and pce in the wrld , Whrs dsrgrd and cntmpt fr hmn<br>
rghts hve rsltd in brbrs acts whch hve outrgd the cnscnce of mnknd ,<br>
and the advnt of a wrld in whch hmn bngs shll enjy frdm of spch and
</div>

Next, let’s combine regular expressions with conditional frequency distributions. Here we will extract all consonant-vowel sequences from the words of Rotokas, such as ka and si. Since each of these is a pair, it can be used to initialize a conditional frequency distribution. We then tabulate the frequency of each pair:

<div class="alert alert-block alert-info">
>>> rotokas_words = nltk.corpus.toolbox.words('rotokas.dic')<br>
>>> cvs = [cv for w in rotokas_words for cv in re.findall(r'[ptksvr][aeiou]', w)]<br>
>>> cfd = nltk.ConditionalFreqDist(cvs)<br>
>>> cfd.tabulate()<br>
     a    e    i    o    u<br>
k  418  148   94  420  173<br>
p   83   31  105   34   51<br>
r  187   63   84   89   79<br>
s    0    0  100    2    1<br>
t   47    8    0  148   37<br>
v   93   27  105   48   49
</div>

Examining the rows for s and t, we see they are in partial “complementary distribution,” which is evidence that they are not distinct phonemes in the language. Thus, we could conceivably drop s from the Rotokas alphabet and simply have a pronunciation rule that the letter t is pronounced s when followed by i. (Note that the single entry having su, namely kasuari, ‘cassowary’ is borrowed from English).

If we want to be able to inspect the words behind the numbers in that table, it would be helpful to have an index, allowing us to quickly find the list of words that contains a given consonant-vowel pair. For example, cv_index['su'] should give us all words containing su. Here’s how we can do this:

<div class="alert alert-block alert-info">
>>> cv_word_pairs = [(cv, w) for w in rotokas_words<br>
...                          for cv in re.findall(r'[ptksvr][aeiou]', w)]<br>
>>> cv_index = nltk.Index(cv_word_pairs)<br>
>>> cv_index['su']<br>
['kasuari']<br>
>>> cv_index['po']<br>
['kaapo', 'kaapopato', 'kaipori', 'kaiporipie', 'kaiporivira', 'kapo', 'kapoa',<br>
'kapokao', 'kapokapo', 'kapokapo', 'kapokapoa', 'kapokapoa', 'kapokapora', ...]
</div>

This program processes each word w in turn, and for each one, finds every substring that matches the regular expression «[ptksvr][aeiou]». In the case of the word kasuari, it finds ka, su, and ri. Therefore, the cv_word_pairs list will contain ('ka', 'kasuari'), ('su', 'kasuari'), and ('ri', 'kasuari'). One further step, using nltk.Index(), converts this into a useful index.

### Finding Word Stems
When we use a web search engine, we usually don’t mind (or even notice) if the words in the document differ from our search terms in having different endings. A query for laptops finds documents containing laptop and vice versa. Indeed, laptop and laptops are just two forms of the same dictionary word (or lemma). For some language processing tasks we want to ignore word endings, and just deal with word stems.

There are various ways we can pull out the stem of a word. Here’s a simple-minded approach that just strips off anything that looks like a suffix:

<div class="alert alert-block alert-info">
>>> def stem(word):<br>
...     for suffix in ['ing', 'ly', 'ed', 'ious', 'ies', 'ive', 'es', 's', 'ment']:<br>
...         if word.endswith(suffix):<br>
...             return word[:-len(suffix)]<br>
...     return word
</div>

Although we will ultimately use NLTK’s built-in stemmers, it’s interesting to see how we can use regular expressions for this task. Our first step is to build up a disjunction of all the suffixes. We need to enclose it in parentheses in order to limit the scope of the disjunction.

<div class="alert alert-block alert-info">
>>> re.findall(r'^.*(ing|ly|ed|ious|ies|ive|es|s|ment)$', 'processing')<br>
['ing']
</div>
    
Here, re.findall() just gave us the suffix even though the regular expression matched the entire word. This is because the parentheses have a second function, to select substrings to be extracted. If we want to use the parentheses to specify the scope of the disjunction, but not to select the material to be output, we have to add ?:, which is just one of many arcane subtleties of regular expressions. Here’s the revised version.

<div class="alert alert-block alert-info">
>>> re.findall(r'^.*(?:ing|ly|ed|ious|ies|ive|es|s|ment)$', 'processing')<br>
['processing']
</div>

However, we’d actually like to split the word into stem and suffix. So we should just parenthesize both parts of the regular expression:

<div class="alert alert-block alert-info">
>>> re.findall(r'^(.*)(ing|ly|ed|ious|ies|ive|es|s|ment)$', 'processing')<br>
[('process', 'ing')]
</div>    
    
This looks promising, but still has a problem. Let’s look at a different word, processes:

<div class="alert alert-block alert-info">
>>> re.findall(r'^(.*)(ing|ly|ed|ious|ies|ive|es|s|ment)$', 'processes')<br>
[('processe', 's')]
</div>

The regular expression incorrectly found an -s suffix instead of an -es suffix. This demonstrates another subtlety: the star operator is “greedy” and so the .* part of the expression tries to consume as much of the input as possible. If we use the “non-greedy” version of the star operator, written *?, we get what we want:

<div class="alert alert-block alert-info">
>>> re.findall(r'^(.*?)(ing|ly|ed|ious|ies|ive|es|s|ment)$', 'processes')<br>
[('process', 'es')]
</div>    
    
This works even when we allow an empty suffix, by making the content of the second parentheses optional:

<div class="alert alert-block alert-info">
>>> re.findall(r'^(.*?)(ing|ly|ed|ious|ies|ive|es|s|ment)?$', 'language')<br>
[('language', '')]
</div>

This approach still has many problems (can you spot them?), but we will move on to define a function to perform stemming, and apply it to a whole text:

<div class="alert alert-block alert-info">
>>> def stem(word):<br>
...     regexp = r'^(.*?)(ing|ly|ed|ious|ies|ive|es|s|ment)?$'<br>
...     stem, suffix = re.findall(regexp, word)[0]<br>
...     return stem<br>
...<br>
>>> raw = """DENNIS: Listen, strange women lying in ponds distributing swords<br>
... is no basis for a system of government.  Supreme executive power derives from<br>
... a mandate from the masses, not from some farcical aquatic ceremony."""<br>
>>> tokens = nltk.word_tokenize(raw)<br>
>>> [stem(t) for t in tokens]<br>
['DENNIS', ':', 'Listen', ',', 'strange', 'women', 'ly', 'in', 'pond',<br>
'distribut', 'sword', 'i', 'no', 'basi', 'for', 'a', 'system', 'of', 'govern',<br>
'.', 'Supreme', 'execut', 'power', 'deriv', 'from', 'a', 'mandate', 'from',<br>
'the', 'mass', ',', 'not', 'from', 'some', 'farcical', 'aquatic', 'ceremony', '.']
</div>    
    
Notice that our regular expression removed the s from ponds but also from is and basis. It produced some non-words, such as distribut and deriv, but these are acceptable stems in some applications.

### Searching Tokenized Text
You can use a special kind of regular expression for searching across multiple words in a text (where a text is a list of tokens). For example, "\<a\> \<man\>" finds all instances of a man in the text. The angle brackets are used to mark token boundaries, and any whitespace between the angle brackets is ignored (behaviors that are unique to NLTK’s findall() method for texts). In the following example, we include <.*> 1, which will match any single token, and enclose it in parentheses so only the matched word (e.g., monied) and not the matched phrase (e.g., a monied man) is produced. The second example finds three-word phrases ending with the word bro 2. The last example finds sequences of three or more words starting with the letter l 3.

<div class="alert alert-block alert-info">
>>> from nltk.corpus import gutenberg, nps_chat<br>
>>> moby = nltk.Text(gutenberg.words('melville-moby_dick.txt'))<br>
>>> moby.findall(r"<a> (<.*>) <man>") 1<br>
monied; nervous; dangerous; white; white; white; pious; queer; good;<br>
mature; white; Cape; great; wise; wise; butterless; white; fiendish;<br>
pale; furious; better; certain; complete; dismasted; younger; brave;<br>
brave; brave; brave<br>
>>> chat = nltk.Text(nps_chat.words())<br>
>>> chat.findall(r"<.*> <.*> <bro>") 2<br>
you rule bro; telling you bro; u twizted bro<br>
>>> chat.findall(r"<l.*>{3,}") 3<br>
lol lol lol; lmao lol lol; lol lol lol; la la la la la; la la la; la<br>
la la; lovely lol lol love; lol lol lol.; la la la; la la la
</div>
    
***    
    NOTE
        Your Turn: Consolidate your understanding of regular expression patterns and 
        substitutions using nltk.re_show(p, s), which annotates the string s to show 
        every place where pattern p was matched, and nltk.app.nemo(), which provides 
        a graphical interface for exploring regular expressions. For more practice, 
        try some of the exercises on regular expressions at the end of this chapter.
***

It is easy to build search patterns when the linguistic phenomenon we’re studying is tied to particular words. In some cases, a little creativity will go a long way. For instance, searching a large text corpus for expressions of the form x and other ys allows us to discover hypernyms (see WordNet):

<div class="alert alert-block alert-info">
>>> from nltk.corpus import brown<br>
>>> hobbies_learned = nltk.Text(brown.words(categories=['hobbies', 'learned']))<br>
>>> hobbies_learned.findall(r"<\w*> <and> <other> <\w*s>")<br>
speed and other activities; water and other liquids; tomb and other<br>
landmarks; Statues and other monuments; pearls and other jewels;<br>
charts and other items; roads and other features; figures and other<br>
objects; military and other areas; demands and other factors;<br>
abstracts and other compilations; iron and other metals
</div>    
    
With enough text, this approach would give us a useful store of information about the taxonomy of objects, without the need for any manual labor. However, our search results will usually contain false positives, i.e., cases that we would want to exclude. For example, the result demands and other factors suggests that demand is an instance of the type factor, but this sentence is actually about wage demands. Nevertheless, we could construct our own ontology of English concepts by manually correcting the output of such searches.

***
    NOTE
        This combination of automatic and manual processing is the most common way 
        for new corpora to be constructed. We will return to this in Chapter 11.
***

Searching corpora also suffers from the problem of false negatives, i.e., omitting cases that we would want to include. It is risky to conclude that some linguistic phenomenon doesn’t exist in a corpus just because we couldn’t find any instances of a search pattern. Perhaps we just didn’t think carefully enough about suitable patterns.

***
    NOTE
        Your Turn: Look for instances of the pattern as x as y to discover 
        information about entities and their properties.
***

## Normalizing Text
In earlier program examples we have often converted text to lowercase before doing anything with its words, e.g., set(w.lower() for w in text). By using lower(), we have normalized the text to lowercase so that the distinction between The and the is ignored. Often we want to go further than this and strip off any affixes, a task known as stemming. A further step is to make sure that the resulting form is a known word in a dictionary, a task known as lemmatization. We discuss each of these in turn. First, we need to define the data we will use in this section:

<div class="alert alert-block alert-info">
>>> raw = """DENNIS: Listen, strange women lying in ponds distributing swords<br>
... is no basis for a system of government.  Supreme executive power derives from<br>
... a mandate from the masses, not from some farcical aquatic ceremony."""<br>
>>> tokens = nltk.word_tokenize(raw)
</div>

### Stemmers
NLTK includes several off-the-shelf stemmers, and if you ever need a stemmer, you should use one of these in preference to crafting your own using regular expressions, since NLTK’s stemmers handle a wide range of irregular cases. The Porter and Lancaster stemmers follow their own rules for stripping affixes. Observe that the Porter stemmer correctly handles the word lying (mapping it to lie), whereas the Lancaster stemmer does not.

<div class="alert alert-block alert-info">
>>> porter = nltk.PorterStemmer()<br>
>>> lancaster = nltk.LancasterStemmer()<br>
>>> [porter.stem(t) for t in tokens]<br>
['DENNI', ':', 'Listen', ',', 'strang', 'women', 'lie', 'in', 'pond',<br>
'distribut', 'sword', 'is', 'no', 'basi', 'for', 'a', 'system', 'of', 'govern',<br>
'.', 'Suprem', 'execut', 'power', 'deriv', 'from', 'a', 'mandat', 'from',<br>
'the', 'mass', ',', 'not', 'from', 'some', 'farcic', 'aquat', 'ceremoni', '.']<br>
>>> [lancaster.stem(t) for t in tokens]<br>
['den', ':', 'list', ',', 'strange', 'wom', 'lying', 'in', 'pond', 'distribut',<br>
'sword', 'is', 'no', 'bas', 'for', 'a', 'system', 'of', 'govern', '.', 'suprem',<br>
'execut', 'pow', 'der', 'from', 'a', 'mand', 'from', 'the', 'mass', ',', 'not',<br>
'from', 'som', 'farc', 'aqu', 'ceremony', '.']
</div>
    
Stemming is not a well-defined process, and we typically pick the stemmer that best suits the application we have in mind. The Porter Stemmer is a good choice if you are indexing some texts and want to support search using alternative forms of words (illustrated in Example 3-1, which uses object-oriented programming techniques that are outside the scope of this book, string formatting techniques to be covered in Formatting: From Lists to Strings, and the enumerate() function to be explained in Sequences).

<i>Example 3-1. Indexing a text using a stemmer.</i>

<div class="alert alert-block alert-info">
class IndexedText(object):<br>
<br>
def __init__(self, stemmer, text):<br>
    self._text = text<br>
    self._stemmer = stemmer<br>
    self._index = nltk.Index((self._stem(word), i)<br>
                             for (i, word) in enumerate(text))<br>
<br>
def concordance(self, word, width=40):<br>
    key = self._stem(word)<br>
    wc = width/4                # words of context<br>
    for i in self._index[key]:<br>
        lcontext = ' '.join(self._text[i-wc:i])<br>
        rcontext = ' '.join(self._text[i:i+wc])<br>
        ldisplay = '%*s'  % (width, lcontext[-width:])<br>
        rdisplay = '%-*s' % (width, rcontext[:width])<br>
        print ldisplay, rdisplay<br>
<br>
def _stem(self, word):<br>
    return self._stemmer.stem(word).lower()
</div>
    
<div class="alert alert-block alert-info">
>>> porter = nltk.PorterStemmer()<br>
>>> grail = nltk.corpus.webtext.words('grail.txt')<br>
>>> text = IndexedText(porter, grail)<br>
>>> text.concordance('lie')<br>
r king ! DENNIS : Listen , strange women lying in ponds distributing swords is no<br>
 beat a very brave retreat . ROBIN : All lies ! MINSTREL : [ singing ] Bravest of<br>
       Nay . Nay . Come . Come . You may lie here . Oh , but you are wounded !<br>
doctors immediately ! No , no , please ! Lie down . [ clap clap ] PIGLET : Well<br>
ere is much danger , for beyond the cave lies the Gorge of Eternal Peril , which<br>
   you . Oh ... TIM : To the north there lies a cave -- the cave of Caerbannog --<br>
h it and lived ! Bones of full fifty men lie strewn about its lair . So , brave k<br>
not stop our fight ' til each one of you lies dead , and the Holy Grail returns t<br>
</div>
    
### Lemmatization
The WordNet lemmatizer removes affixes only if the resulting word is in its dictionary. This additional checking process makes the lemmatizer slower than the stemmers just mentioned. Notice that it doesn’t handle lying, but it converts women to woman.

<div class="alert alert-block alert-info">
>>> wnl = nltk.WordNetLemmatizer()<br>
>>> [wnl.lemmatize(t) for t in tokens]<br>
['DENNIS', ':', 'Listen', ',', 'strange', 'woman', 'lying', 'in', 'pond',<br>
'distributing', 'sword', 'is', 'no', 'basis', 'for', 'a', 'system', 'of',<br>
'government', '.', 'Supreme', 'executive', 'power', 'derives', 'from', 'a',<br>
'mandate', 'from', 'the', 'mass', ',', 'not', 'from', 'some', 'farcical',<br>
'aquatic', 'ceremony', '.']
</div>
    
The WordNet lemmatizer is a good choice if you want to compile the vocabulary of some texts and want a list of valid lemmas (or lexicon headwords).

***    
    NOTE
        Another normalization task involves identifying non-standard words, including numbers, 
        abbreviations, and dates, and mapping any such tokens to a special vocabulary. 
        For example, every decimal number could be mapped to a single token 0.0, and every 
        acronym could be mapped to AAA. This keeps the vocabulary small and improves the 
        accuracy of many language modeling tasks.
***    

## Regular Expressions for Tokenizing Text
Tokenization is the task of cutting a string into identifiable linguistic units that constitute a piece of language data. Although it is a fundamental task, we have been able to delay it until now because many corpora are already tokenized, and because NLTK includes some tokenizers. Now that you are familiar with regular expressions, you can learn how to use them to tokenize text, and to have much more control over the process.

### Simple Approaches to Tokenization
The very simplest method for tokenizing text is to split on whitespace. Consider the following text from Alice’s Adventures in Wonderland:

<div class="alert alert-block alert-info">
>>> raw = """'When I'M a Duchess,' she said to herself, (not in a very hopeful tone<br>
... though), 'I won't have any pepper in my kitchen AT ALL. Soup does very<br>
... well without--Maybe it's always pepper that makes people hot-tempered,'..."""
</div>
    
We could split this raw text on whitespace using raw.split(). To do the same using a regular expression, it is not enough to match any space characters in the string , since this results in tokens that contain a \n newline character; instead, we need to match any number of spaces, tabs, or newlines :

<div class="alert alert-block alert-info">
>>> re.split(r' ', raw) <br>
["'When", "I'M", 'a', "Duchess,'", 'she', 'said', 'to', 'herself,', '(not', 'in',<br>
'a', 'very', 'hopeful', 'tone\nthough),', "'I", "won't", 'have', 'any', 'pepper',<br>
'in', 'my', 'kitchen', 'AT', 'ALL.', 'Soup', 'does', 'very\nwell', 'without--Maybe',<br>
"it's", 'always', 'pepper', 'that', 'makes', 'people', "hot-tempered,'..."]<br>
>>> re.split(r'[ \t\n]+', raw) <br>
["'When", "I'M", 'a', "Duchess,'", 'she', 'said', 'to', 'herself,', '(not', 'in',<br>
'a', 'very', 'hopeful', 'tone', 'though),', "'I", "won't", 'have', 'any', 'pepper',<br>
'in', 'my', 'kitchen', 'AT', 'ALL.', 'Soup', 'does', 'very', 'well', 'without--Maybe',<br>
"it's", 'always', 'pepper', 'that', 'makes', 'people', "hot-tempered,'..."]
</div>    
    
The regular expression «[ \t\n]+» matches one or more spaces, tabs (\t), or newlines (\n). Other whitespace characters, such as carriage return and form feed, should really be included too. Instead, we will use a built-in re abbreviation, \s, which means any whitespace character. The second statement in the preceding example can be rewritten as re.split(r'\s+', raw).

***    
    NOTE
        Important: Remember to prefix regular expressions with the letter r (meaning “raw”), 
        which instructs the Python interpreter to treat the string literally, 
        rather than processing any backslashed characters it contains.
***    

Splitting on whitespace gives us tokens like '(not' and 'herself,'. An alternative is to use the fact that Python provides us with a character class \w for word characters, equivalent to [a-zA-Z0-9_]. It also defines the complement of this class, \W, i.e., all characters other than letters, digits, or underscore. We can use \W in a simple regular expression to split the input on anything other than a word character:

<div class="alert alert-block alert-info">
>>> re.split(r'\W+', raw)<br>
['', 'When', 'I', 'M', 'a', 'Duchess', 'she', 'said', 'to', 'herself', 'not', 'in',<br>
'a', 'very', 'hopeful', 'tone', 'though', 'I', 'won', 't', 'have', 'any', 'pepper',<br>
'in', 'my', 'kitchen', 'AT', 'ALL', 'Soup', 'does', 'very', 'well', 'without',<br>
'Maybe', 'it', 's', 'always', 'pepper', 'that', 'makes', 'people', 'hot', 'tempered',<br>
'']
</div> 
    
Observe that this gives us empty strings at the start and the end (to understand why, try doing 'xx'.split('x')). With re.findall(r'\w+', raw), we get the same tokens, but without the empty strings, using a pattern that matches the words instead of the spaces. Now that we’re matching the words, we’re in a position to extend the regular expression to cover a wider range of cases. The regular expression «\w+|\S\w*» will first try to match any sequence of word characters. If no match is found, it will try to match any non-whitespace character (\S is the complement of \s) followed by further word characters. This means that punctuation is grouped with any following letters (e.g., ’s) but that sequences of two or more punctuation characters are separated.

<div class="alert alert-block alert-info">
>>> re.findall(r'\w+|\S\w*', raw)<br>
["'When", 'I', "'M", 'a', 'Duchess', ',', "'", 'she', 'said', 'to', 'herself', ',',<br>
'(not', 'in', 'a', 'very', 'hopeful', 'tone', 'though', ')', ',', "'I", 'won', "'t",<br>
'have', 'any', 'pepper', 'in', 'my', 'kitchen', 'AT', 'ALL', '.', 'Soup', 'does',<br>
'very', 'well', 'without', '-', '-Maybe', 'it', "'s", 'always', 'pepper', 'that',<br>
'makes', 'people', 'hot', '-tempered', ',', "'", '.', '.', '.']
</div>
    
Let’s generalize the \w+ in the preceding expression to permit word-internal hyphens and apostrophes: «\w+([-']\w+)*». This expression means \w+ followed by zero or more instances of [-']\w+; it would match hot-tempered and it’s. (We need to include ?: in this expression for reasons discussed earlier.) We’ll also add a pattern to match quote characters so these are kept separate from the text they enclose.

<div class="alert alert-block alert-info">
>>> print re.findall(r"\w+(?:[-']\w+)*|'|[-.(]+|\S\w*", raw)<br>
["'", 'When', "I'M", 'a', 'Duchess', ',', "'", 'she', 'said', 'to', 'herself', ',',<br>
'(', 'not', 'in', 'a', 'very', 'hopeful', 'tone', 'though', ')', ',', "'", 'I',<br>
"won't", 'have', 'any', 'pepper', 'in', 'my', 'kitchen', 'AT', 'ALL', '.', 'Soup',<br>
'does', 'very', 'well', 'without', '--', 'Maybe', "it's", 'always', 'pepper',<br>
'that', 'makes', 'people', 'hot-tempered', ',', "'", '...']
</div>    
    
The expression in this example also included «[-.(]+», which causes the double hyphen, ellipsis, and open parenthesis to be tokenized separately.

Table 3-4 lists the regular expression character class symbols we have seen in this section, in addition to some other useful symbols.
![table 3-4](regexSym.png)
    <i>Table 3-4. Regular expression symbols</i>

### NLTK’s Regular Expression Tokenizer
The function nltk.regexp_tokenize() is similar to re.findall() (as we’ve been using it for tokenization). However, nltk.regexp_tokenize() is more efficient for this task, and avoids the need for special treatment of parentheses. For readability we break up the regular expression over several lines and add a comment about each line. The special (?x) “verbose flag” tells Python to strip out the embedded whitespace and comments.

<div class="alert alert-block alert-info">
>>> text = 'That U.S.A. poster-print costs \$12.40...'<br>
>>> pattern = r'''(?x)    # set flag to allow verbose regexps<br>
...     ([A-Z]\.)+        # abbreviations, e.g. U.S.A.<br>
...   | \w+(-\w+)*        # words with optional internal hyphens<br>
...   | \$?\d+(\.\d+)?%?  # currency and percentages, e.g. \$12.40, 82%<br>
...   | \.\.\.            # ellipsis<br>
...   | [][.,;"'?():-_`]  # these are separate tokens<br>
... '''<br>
>>> nltk.regexp_tokenize(text, pattern)<br>
['That', 'U.S.A.', 'poster-print', 'costs', '\$12.40', '...']
</div>    
    
When using the verbose flag, you can no longer use ' ' to match a space character; use \s instead. The regexp_tokenize() function has an optional gaps parameter. When set to True, the regular expression specifies the gaps between tokens, as with re.split().

***    
    NOTE
        We can evaluate a tokenizer by comparing the resulting tokens with a wordlist, 
        and then report any tokens that don’t appear in the wordlist, using set(tokens).difference(wordlist). 
        You’ll probably want to lowercase all the tokens first.
***    

### Further Issues with Tokenization
Tokenization turns out to be a far more difficult task than you might have expected. No single solution works well across the board, and we must decide what counts as a token depending on the application domain.

When developing a tokenizer it helps to have access to raw text which has been manually tokenized, in order to compare the output of your tokenizer with high-quality (or “gold-standard”) tokens. The NLTK corpus collection includes a sample of Penn Treebank data, including the raw Wall Street Journal text (nltk.corpus.treebank_raw.raw()) and the tokenized version (nltk.corpus.treebank.words()).

A final issue for tokenization is the presence of contractions, such as didn’t. If we are analyzing the meaning of a sentence, it would probably be more useful to normalize this form to two separate forms: did and n’t (or not). We can do this work with the help of a lookup table.

## Segmentation
This section discusses more advanced concepts, which you may prefer to skip on the first time through this chapter.

Tokenization is an instance of a more general problem of segmentation. In this section, we will look at two other instances of this problem, which use radically different techniques to the ones we have seen so far in this chapter.

### Sentence Segmentation
Manipulating texts at the level of individual words often presupposes the ability to divide a text into individual sentences. As we have seen, some corpora already provide access at the sentence level. In the following example, we compute the average number of words per sentence in the Brown Corpus:

<div class="alert alert-block alert-info">
>>> len(nltk.corpus.brown.words()) / len(nltk.corpus.brown.sents())<br>
20.250994070456922
</div>
    
In other cases, the text is available only as a stream of characters. Before tokenizing the text into words, we need to segment it into sentences. NLTK facilitates this by including the Punkt sentence segmenter (Kiss & Strunk, 2006). Here is an example of its use in segmenting the text of a novel. (Note that if the segmenter’s internal data has been updated by the time you read this, you will see different output.)

<div class="alert alert-block alert-info">
>>> sent_tokenizer=nltk.data.load('tokenizers/punkt/english.pickle')<br>
>>> text = nltk.corpus.gutenberg.raw('chesterton-thursday.txt')<br>
>>> sents = sent_tokenizer.tokenize(text)<br>
>>> pprint.pprint(sents[171:181])<br>
['"Nonsense!',<br>
 '" said Gregory, who was very rational when anyone else\nattempted paradox.',<br>
 '"Why do all the clerks and navvies in the\nrailway trains look so sad and tired,...',<br>
 'I will\ntell you.',<br>
 'It is because they know that the train is going right.',<br>
 'It\nis because they know that whatever place they have taken a ticket\nfor that ...',<br>
 'It is because after they have\npassed Sloane Square they know that the next stat...',<br>
 'Oh, their wild rapture!',<br>
 'oh,\ntheir eyes like stars and their souls again in Eden, if the next\nstation w...'<br>
 '"\n\n"It is you who are unpoetical," replied the poet Syme.']
</div>    
    
Notice that this example is really a single sentence, reporting the speech of Mr. Lucian Gregory. However, the quoted speech contains several sentences, and these have been split into individual strings. This is reasonable behavior for most applications.

Sentence segmentation is difficult because a period is used to mark abbreviations, and some periods simultaneously mark an abbreviation and terminate a sentence, as often happens with acronyms like U.S.A.

For another approach to sentence segmentation, see Further Examples of Supervised Classification.

### Word Segmentation
For some writing systems, tokenizing text is made more difficult by the fact that there is no visual representation of word boundaries. For example, in Chinese, the three-character string: 爱国人 (ai4 “love” [verb], guo3 “country”, ren2 “person”) could be tokenized as 爱国 / 人, “country-loving person,” or as 爱 / 国人, “love country-person.”

A similar problem arises in the processing of spoken language, where the hearer must segment a continuous speech stream into individual words. A particularly challenging version of this problem arises when we don’t know the words in advance. This is the problem faced by a language learner, such as a child hearing utterances from a parent. Consider the following artificial example, where word boundaries have been removed:

Example 3-2. 
<ol>
<li>doyouseethekitty</li>

<li>seethedoggy</li>

<li>doyoulikethekitty</li>

<li>likethedoggy</li>
</ol>

Our first challenge is simply to represent the problem: we need to find a way to separate text content from the segmentation. We can do this by annotating each character with a boolean value to indicate whether or not a word-break appears after the character (an idea that will be used heavily for “chunking” in Chapter 7). Let’s assume that the learner is given the utterance breaks, since these often correspond to extended pauses. Here is a possible representation, including the initial and target segmentations:

<div class="alert alert-block alert-info">
>>> text = "doyouseethekittyseethedoggydoyoulikethekittylikethedoggy"<br>
>>> seg1 = "0000000000000001000000000010000000000000000100000000000"<br>
>>> seg2 = "0100100100100001001001000010100100010010000100010010000"<br>
</div>    
Observe that the segmentation strings consist of zeros and ones. They are one character shorter than the source text, since a text of length n can be broken up in only n–1 places. The segment() function in Example 3-3 demonstrates that we can get back to the original segmented text from its representation.

<i>Example 3-3. Reconstruct segmented text from string representation: seg1 and seg2 represent the initial and final segmentations of some hypothetical child-directed speech; the segment() function can use them to reproduce the segmented text.</i>

<div class="alert alert-block alert-info">
def segment(text, segs):<br>
    words = []<br>
    last = 0<br>
    for i in range(len(segs)):<br>
        if segs[i] == '1':<br>
            words.append(text[last:i+1])<br>
            last = i+1<br>
    words.append(text[last:])<br>
    return words<br>
>>> text = "doyouseethekittyseethedoggydoyoulikethekittylikethedoggy"<br>
>>> seg1 = "0000000000000001000000000010000000000000000100000000000"<br>
>>> seg2 = "0100100100100001001001000010100100010010000100010010000"<br>
>>> segment(text, seg1)<br>
['doyouseethekitty', 'seethedoggy', 'doyoulikethekitty', 'likethedoggy']<br>
>>> segment(text, seg2)<br>
['do', 'you', 'see', 'the', 'kitty', 'see', 'the', 'doggy', 'do', 'you',<br>
 'like', 'the', kitty', 'like', 'the', 'doggy']
</div>    
    
Now the segmentation task becomes a search problem: find the bit string that causes the text string to be correctly segmented into words. We assume the learner is acquiring words and storing them in an internal lexicon. Given a suitable lexicon, it is possible to reconstruct the source text as a sequence of lexical items. Following (Brent & Cartwright, 1995), we can define an objective function, a scoring function whose value we will try to optimize, based on the size of the lexicon and the amount of information needed to reconstruct the source text from the lexicon. We illustrate this in Figure 3-6.
    
![figure 3-6](segmentation.png)

<i>Figure 3-6. Calculation of objective function: Given a hypothetical segmentation of the source text (on the left), derive a lexicon and a derivation table that permit the source text to be reconstructed, then total up the number of characters used by each lexical item (including a boundary marker) and each derivation, to serve as a score of the quality of the segmentation; smaller values of the score indicate a better segmentation.</i>

It is a simple matter to implement this objective function, as shown in Example 3-4.

<i>Example 3-4. Computing the cost of storing the lexicon and reconstructing the source text.</i>

<div class="alert alert-block alert-info">
def evaluate(text, segs):<br>
    words = segment(text, segs)<br>
    text_size = len(words)<br>
    lexicon_size = len(' '.join(list(set(words))))<br>
    return text_size + lexicon_size
</div>
    
<div class="alert alert-block alert-info">
>>> text = "doyouseethekittyseethedoggydoyoulikethekittylikethedoggy"<br>
>>> seg1 = "0000000000000001000000000010000000000000000100000000000"<br>
>>> seg2 = "0100100100100001001001000010100100010010000100010010000"<br>
>>> seg3 = "0000100100000011001000000110000100010000001100010000001"<br>
>>> segment(text, seg3)<br>
['doyou', 'see', 'thekitt', 'y', 'see', 'thedogg', 'y', 'doyou', 'like',<br>
 'thekitt', 'y', 'like', 'thedogg', 'y']<br>
>>> evaluate(text, seg3)<br>
46<br>
>>> evaluate(text, seg2)<br>
47<br>
>>> evaluate(text, seg1)<br>
63<br>
</div>

The final step is to search for the pattern of zeros and ones that minimizes this objective function, shown in Example 3-5. Notice that the best segmentation includes “words” like thekitty, since there’s not enough evidence in the data to split this any further.

<i>Example 3-5. Non-deterministic search using simulated annealing: Begin searching with phrase segmentations only; randomly perturb the zeros and ones proportional to the “temperature”; with each iteration the temperature is lowered and the perturbation of boundaries is reduced.</i>

<div class="alert alert-block alert-info">
from random import randint<br>
<br>
def flip(segs, pos):<br>
    return segs[:pos] + str(1-int(segs[pos])) + segs[pos+1:] <br>
<br>
def flip_n(segs, n):<br>
    for i in range(n):<br>
        segs = flip(segs, randint(0,len(segs)-1))<br>
    return segs<br>
<br>
def anneal(text, segs, iterations, cooling_rate):<br>
    temperature = float(len(segs))<br>
    while temperature > 0.5:<br>
        best_segs, best = segs, evaluate(text, segs)<br>
        for i in range(iterations):<br>
            guess = flip_n(segs, int(round(temperature)))<br>
            score = evaluate(text, guess)<br>
            if score < best:<br>
                best, best_segs = score, guess<br>
        score, segs = best, best_segs<br>
        temperature = temperature / cooling_rate<br>
        print evaluate(text, segs), segment(text, segs)<br>
    print<br>
    return segs
</div>
                            
<div class="alert alert-block alert-info">
>>> text = "doyouseethekittyseethedoggydoyoulikethekittylikethedoggy"<br>
>>> seg1 = "0000000000000001000000000010000000000000000100000000000"<br>
>>> anneal(text, seg1, 5000, 1.2)<br>
60 ['doyouseetheki', 'tty', 'see', 'thedoggy', 'doyouliketh', 'ekittylike', 'thedoggy']<br>
58 ['doy', 'ouseetheki', 'ttysee', 'thedoggy', 'doy', 'o', 'ulikethekittylike', 'thedoggy']<br>
56 ['doyou', 'seetheki', 'ttysee', 'thedoggy', 'doyou', 'liketh', 'ekittylike', 'thedoggy']<br>
54 ['doyou', 'seethekit', 'tysee', 'thedoggy', 'doyou', 'likethekittylike', 'thedoggy']<br>
53 ['doyou', 'seethekit', 'tysee', 'thedoggy', 'doyou', 'like', 'thekitty', 'like', 'thedoggy']<br>
51 ['doyou', 'seethekittysee', 'thedoggy', 'doyou', 'like', 'thekitty', 'like', 'thedoggy']<br>
42 ['doyou', 'see', 'thekitty', 'see', 'thedoggy', 'doyou', 'like', 'thekitty', 'like', 'thedoggy']<br>
'0000100100000001001000000010000100010000000100010000000'
</div>
    
With enough data, it is possible to automatically segment text into words with a reasonable degree of accuracy. Such methods can be applied to tokenization for writing systems that don’t have any visual representation of word boundaries.

## Formatting: From Lists to Strings
Often we write a program to report a single data item, such as a particular element in a corpus that meets some complicated criterion, or a single summary statistic such as a word-count or the performance of a tagger. More often, we write a program to produce a structured result; for example, a tabulation of numbers or linguistic forms, or a reformatting of the original data. When the results to be presented are linguistic, textual output is usually the most natural choice. However, when the results are numerical, it may be preferable to produce graphical output. In this section, you will learn about a variety of ways to present program output.

### From Lists to Strings
The simplest kind of structured object we use for text processing is lists of words. When we want to output these to a display or a file, we must convert these lists into strings. To do this in Python we use the join() method, and specify the string to be used as the “glue”:

<div class="alert alert-block alert-info">
>>> silly = ['We', 'called', 'him', 'Tortoise', 'because', 'he', 'taught', 'us', '.']<br>
>>> ' '.join(silly)<br>
'We called him Tortoise because he taught us .'<br>
>>> ';'.join(silly)<br>
'We;called;him;Tortoise;because;he;taught;us;.'<br>
>>> ''.join(silly)<br>
'WecalledhimTortoisebecausehetaughtus.'<br>
</div> 
    
So ' '.join(silly) means: take all the items in silly and concatenate them as one big string, using ' ' as a spacer between the items. I.e., join() is a method of the string that you want to use as the glue. (Many people find this notation for join() counter-intuitive.) The join() method only works on a list of strings—what we have been calling a text—a complex type that enjoys some privileges in Python.

### Strings and Formats
We have seen that there are two ways to display the contents of an object:

<div class="alert alert-block alert-info">
>>> word = 'cat'<br>
>>> sentence = """hello<br>
... world"""<br>
>>> print word<br>
cat<br>
>>> print sentence<br>
hello<br>
world<br>
>>> word<br>
'cat'<br>
>>> sentence<br>
'hello\nworld'
</div> 
    
The print command yields Python’s attempt to produce the most human-readable form of an object. The second method—naming the variable at a prompt—shows us a string that can be used to recreate this object. It is important to keep in mind that both of these are just strings, displayed for the benefit of you, the user. They do not give us any clue as to the actual internal representation of the object.

There are many other useful ways to display an object as a string of characters. This may be for the benefit of a human reader, or because we want to export our data to a particular file format for use in an external program.

Formatted output typically contains a combination of variables and pre-specified strings. For example, given a frequency distribution fdist, we could do:

<div class="alert alert-block alert-info">
>>> fdist = nltk.FreqDist(['dog', 'cat', 'dog', 'cat', 'dog', 'snake', 'dog', 'cat'])<br>
>>> for word in fdist:<br>
...     print word, '->', fdist[word], ';',<br>
dog -> 4 ; cat -> 3 ; snake -> 1 ;
</div>
    
Apart from the problem of unwanted whitespace, print statements that contain alternating variables and constants can be difficult to read and maintain. A better solution is to use string formatting expressions.

<div class="alert alert-block alert-info">
>>> for word in fdist:<br>
...    print '%s->%d;' % (word, fdist[word]),<br>
dog->4; cat->3; snake->1;
</div>
    
To understand what is going on here, let’s test out the string formatting expression on its own. (By now this will be your usual method of exploring new syntax.)

<div class="alert alert-block alert-info">
>>> '%s->%d;' % ('cat', 3)<br>
'cat->3;'<br>
>>> '%s->%d;' % 'cat'<br>
Traceback (most recent call last):<br>
  File "<stdin>", line 1, in <module><br>
TypeError: not enough arguments for format string<br>
</div>
    
The special symbols %s and %d are placeholders for strings and (decimal) integers. We can embed these inside a string, then use the % operator to combine them. Let’s unpack this code further, in order to see this behavior up close:

<div class="alert alert-block alert-info">
>>> '%s->' % 'cat'<br>
'cat->'<br>
>>> '%d' % 3<br>
'3'<br>
>>> 'I want a %s right now' % 'coffee'<br>
'I want a coffee right now'<br>
</div>
    
We can have a number of placeholders, but following the % operator we need to specify a tuple with exactly the same number of values:

<div class="alert alert-block alert-info">
>>> "%s wants a %s %s" % ("Lee", "sandwich", "for lunch")<br>
'Lee wants a sandwich for lunch'
</div> 
    
We can also provide the values for the placeholders indirectly. Here’s an example using a for loop:

<div class="alert alert-block alert-info">
>>> template = 'Lee wants a %s right now'<br>
>>> menu = ['sandwich', 'spam fritter', 'pancake']<br>
>>> for snack in menu:<br>
...     print template % snack<br>
...<br>
Lee wants a sandwich right now<br>
Lee wants a spam fritter right now<br>
Lee wants a pancake right now
</div> 
    
The %s and %d symbols are called conversion specifiers. They start with the % character and end with a conversion character such as s (for string) or d (for decimal integer) The string containing conversion specifiers is called a format string. We combine a format string with the % operator and a tuple of values to create a complete string formatting expression.

### Lining Things Up
So far our formatting strings generated output of arbitrary width on the page (or screen), such as %s and %d. We can specify a width as well, such as %6s, producing a string that is padded to width 6. It is right-justified by default 1, but we can include a minus sign to make it left-justified 2. In case we don’t know in advance how wide a displayed value should be, the width value can be replaced with a star in the formatting string, then specified using a variable 3.

<div class="alert alert-block alert-info">
>>> '%6s' % 'dog' 1<br>
'   dog'<br>
>>> '%-6s' % 'dog' 2<br>
'dog   '<br>
>>> width = 6<br>
>>> '%-*s' % (width, 'dog') 3<br>
'dog   '
</div>    
    
Other control characters are used for decimal integers and floating-point numbers. Since the percent character % has a special interpretation in formatting strings, we have to precede it with another % to get it in the output.

<div class="alert alert-block alert-info">
>>> count, total = 3205, 9375<br>
>>> "accuracy for %d words: %2.4f%%" % (total, 100 * count / total)<br>
'accuracy for 9375 words: 34.1867%'
</div>
    
An important use of formatting strings is for tabulating data. Recall that in Accessing Text Corpora we saw data being tabulated from a conditional frequency distribution. Let’s perform the tabulation ourselves, exercising full control of headings and column widths, as shown in Example 3-6. Note the clear separation between the language processing work, and the tabulation of results.

<i>Example 3-6. Frequency of modals in different sections of the Brown Corpus.</i>

<div class="alert alert-block alert-info">
def tabulate(cfdist, words, categories):<br>
    print '%-16s' % 'Category',<br>
    for word in words:                                  # column headings<br>
        print '%6s' % word,<br>
    print<br>
    for category in categories:<br>
        print '%-16s' % category,                       # row heading<br>
        for word in words:                              # for each word<br>
            print '%6d' % cfdist[category][word],       # print table cell<br>
        print                                           # end the row<br>
</div>
    
<div class="alert alert-block alert-info">
>>> from nltk.corpus import brown<br>
>>> cfd = nltk.ConditionalFreqDist(<br>
...           (genre, word)<br>
...           for genre in brown.categories()<br>
...           for word in brown.words(categories=genre))<br>
>>> genres = ['news', 'religion', 'hobbies', 'science_fiction', 'romance', 'humor']<br>
>>> modals = ['can', 'could', 'may', 'might', 'must', 'will']<br>
>>> tabulate(cfd, modals, genres)<br>
Category            can  could    may  might   must   will<br>
news                 93     86     66     38     50    389<br>
religion             82     59     78     12     54     71<br>
hobbies             268     58    131     22     83    264<br>
science_fiction      16     49      4     12      8     16<br>
romance              74    193     11     51     45     43<br>
humor                16     30      8      8      9     13<br>
</div>
    
Recall from the listing in Example 3-1 that we used a formatting string "%*s". This allows us to specify the width of a field using a variable.

<div class="alert alert-block alert-info">
>>> '%*s' % (15, "Monty Python")<br>
'   Monty Python'
</div>
    
We could use this to automatically customize the column to be just wide enough to accommodate all the words, using width = max(len(w) for w in words). Remember that the comma at the end of print statements adds an extra space, and this is sufficient to prevent the column headings from running into each other.

### Writing Results to a File
We have seen how to read text from files (Accessing Text from the Web and from Disk). It is often useful to write output to files as well. The following code opens a file output.txt for writing, and saves the program output to the file.

<div class="alert alert-block alert-info">
>>> output_file = open('output.txt', 'w')<br>
>>> words = set(nltk.corpus.genesis.words('english-kjv.txt'))<br>
>>> for word in sorted(words):<br>
...     output_file.write(word + "\n")
</div>
    
***    
    NOTE
        Your Turn: What is the effect of appending \n to each string before we write it to the file? 
        If you’re using a Windows machine, you may want to use word + "\r\n" instead. What happens if we do?
    
<div class="alert alert-block alert-info">
output_file.write(word)
</div>
    
***    
    
When we write non-text data to a file, we must convert it to a string first. We can do this conversion using formatting strings, as we saw earlier. Let’s write the total number of words to our file, before closing it.

<div class="alert alert-block alert-info">
>>> len(words)<br>
2789<br>
>>> str(len(words))<br>
'2789'<br>
>>> output_file.write(str(len(words)) + "\n")<br>
>>> output_file.close()
</div>
    
<div class="alert alert-block alert-danger">
    <b>Caution!</b>
You should avoid filenames that contain space characters, such as output file.txt, or that are identical except for case distinctions, e.g., Output.txt and output.TXT.
</div>
    
### Text Wrapping
When the output of our program is text-like, instead of tabular, it will usually be necessary to wrap it so that it can be displayed conveniently. Consider the following output, which overflows its line, and which uses a complicated print statement:

<div class="alert alert-block alert-info">
>>> saying = ['After', 'all', 'is', 'said', 'and', 'done', ',',<br>
...           'more', 'is', 'said', 'than', 'done', '.']<br>
>>> for word in saying:<br>
...     print word, '(' + str(len(word)) + '),',
</div>
    
After (5), all (3), is (2), said (4), and (3), done (4), , (1), more (4), is (2), said (4), 
We can take care of line wrapping with the help of Python’s textwrap module. For maximum clarity we will separate each step onto its own line:

<div class="alert alert-block alert-info">
>>> from textwrap import fill<br>
>>> format = '%s (%d),'<br>
>>> pieces = [format % (word, len(word)) for word in saying]<br>
>>> output = ' '.join(pieces)<br>
>>> wrapped = fill(output)<br>
>>> print wrapped<br>
After (5), all (3), is (2), said (4), and (3), done (4), , (1), more<br>
(4), is (2), said (4), than (4), done (4), . (1),
</div>    
    
Notice that there is a linebreak between more and its following number. If we wanted to avoid this, we could redefine the formatting string so that it contained no spaces (e.g., '%s_(%d),'), then instead of printing the value of wrapped, we could print wrapped.replace('_', ' ').

## Summary
In this book we view a text as a list of words. A “raw text” is a potentially long string containing words and whitespace formatting, and is how we typically store and visualize a text.

A string is specified in Python using single or double quotes: 'Monty Python', "Monty Python".

The characters of a string are accessed using indexes, counting from zero: 'Monty Python'[0] gives the value M. The length of a string is found using len().

Substrings are accessed using slice notation: 'Monty Python'[1:5] gives the value onty. If the start index is omitted, the substring begins at the start of the string; if the end index is omitted, the slice continues to the end of the string.

Strings can be split into lists: 'Monty Python'.split() gives ['Monty', 'Python']. Lists can be joined into strings: '/'.join(['Monty', 'Python']) gives 'Monty/Python'.

We can read text from a file f using text = open(f).read(). We can read text from a URL u using text = urlopen(u).read(). We can iterate over the lines of a text file using for line in open(f).

Texts found on the Web may contain unwanted material (such as headers, footers, and markup), that need to be removed before we do any linguistic processing.

Tokenization is the segmentation of a text into basic units—or tokens—such as words and punctuation. Tokenization based on whitespace is inadequate for many applications because it bundles punctuation together with words. NLTK provides an off-the-shelf tokenizer nltk.word_tokenize().

Lemmatization is a process that maps the various forms of a word (such as appeared, appears) to the canonical or citation form of the word, also known as the lexeme or lemma (e.g., appear).

Regular expressions are a powerful and flexible method of specifying patterns. Once we have imported the re module, we can use re.findall() to find all substrings in a string that match a pattern.

If a regular expression string includes a backslash, you should tell Python not to preprocess the string, by using a raw string with an r prefix: r'regexp'.

When backslash is used before certain characters, e.g., \n, this takes on a special meaning (newline character); however, when backslash is used before regular expression wildcards and operators, e.g., \., \|, \$, these characters lose their special meaning and are matched literally.

A string formatting expression template % arg_tuple consists of a format string template that contains conversion specifiers like %-6s and %0.2d.

## Further Reading
Extra materials for this chapter are posted at http://www.nltk.org/, including links to freely available resources on the Web. Remember to consult the Python reference materials at http://docs.python.org/. (For example, this documentation covers “universal newline support,” explaining how to work with the different newline conventions used by various operating systems.)

For more examples of processing words with NLTK, see the tokenization, stemming, and corpus HOWTOs at http://www.nltk.org/howto. Chapters 2 and 3 of (Jurafsky & Martin, 2008) contain more advanced material on regular expressions and morphology. For more extensive discussion of text processing with Python, see (Mertz, 2003). For information about normalizing non-standard words, see (Sproat et al., 2001).

There are many references for regular expressions, both practical and theoretical. For an introductory tutorial to using regular expressions in Python, see Kuchling’s Regular Expression HOWTO, http://www.amk.ca/python/howto/regex/. For a comprehensive and detailed manual in using regular expressions, covering their syntax in most major programming languages, including Python, see (Friedl, 2002). Other presentations include Section 2.1 of (Jurafsky & Martin, 2008), and Chapter 3 of (Mertz, 2003).

There are many online resources for Unicode. Useful discussions of Python’s facilities for handling Unicode are:

PEP-100 http://www.python.org/dev/peps/pep-0100/

Jason Orendorff, Unicode for Programmers, http://www.jorendorff.com/articles/unicode/

A. M. Kuchling, Unicode HOWTO, http://www.amk.ca/python/howto/unicode

Frederik Lundh, Python Unicode Objects, http://effbot.org/zone/unicode-objects.htm

Joel Spolsky, The Absolute Minimum Every Software Developer Absolutely, Positively Must Know About Unicode and Character Sets (No Excuses!), http://www.joelonsoftware.com/articles/Unicode.html

The problem of tokenizing Chinese text is a major focus of SIGHAN, the ACL Special Interest Group on Chinese Language Processing (http://sighan.org/). Our method for segmenting English text follows (Brent & Cartwright, 1995); this work falls in the area of language acquisition (Niyogi, 2006).

Collocations are a special case of multiword expressions. A multiword expression is a small phrase whose meaning and other properties cannot be predicted from its words alone, e.g., part-of-speech (Baldwin & Kim, 2010).

Simulated annealing is a heuristic for finding a good approximation to the optimum value of a function in a large, discrete search space, based on an analogy with annealing in metallurgy. The technique is described in many Artificial Intelligence texts.

The approach to discovering hyponyms in text using search patterns like x and other ys is described by (Hearst, 1992).

## Exercises
○ Define a string s = 'colorless'. Write a Python statement that changes this to “colourless” using only the slice and concatenation operations.

○ We can use the slice notation to remove morphological endings on words. For example, 'dogs'[:-1] removes the last character of dogs, leaving dog. Use slice notation to remove the affixes from these words (we’ve inserted a hyphen to indicate the affix boundary, but omit this from your strings): dish-es, run-ning, nation-ality, un-do, pre-heat.

○ We saw how we can generate an IndexError by indexing beyond the end of a string. Is it possible to construct an index that goes too far to the left, before the start of the string?

○ We can specify a “step” size for the slice. The following returns every second character within the slice: monty[6:11:2]. It also works in the reverse direction: monty[10:5:-2]. Try these for yourself, and then experiment with different step values.

○ What happens if you ask the interpreter to evaluate monty[::-1]? Explain why this is a reasonable result.

○ Describe the class of strings matched by the following regular expressions:
<ul>
<li>[a-zA-Z]+</li>

<li>[A-Z][a-z]*</li>

<li>p[aeiou]{,2}t</li>

<li>\d+(\.\d+)?</li>

<li>([^aeiou][aeiou][^aeiou])*</li>

<li>\w+|[^\w\s]+</li>
</ul>
Test your answers using nltk.re_show().

○ Write regular expressions to match the following classes of strings:

A single determiner (assume that a, an, and the are the only determiners)

An arithmetic expression using integers, addition, and multiplication, such as 2*3+8

○ Write a utility function that takes a URL as its argument, and returns the contents of the URL, with all HTML markup removed. Use urllib.urlopen to access the contents of the URL, e.g.:

raw_contents = urllib.urlopen('http://www.nltk.org/').read()
○ Save some text into a file corpus.txt. Define a function load(f) that reads from the file named in its sole argument, and returns a string containing the text of the file.

Use nltk.regexp_tokenize() to create a tokenizer that tokenizes the various kinds of punctuation in this text. Use one multiline regular expression inline comments, using the verbose flag (?x).

Use nltk.regexp_tokenize() to create a tokenizer that tokenizes the following kinds of expressions: monetary amounts; dates; names of people and organizations.

○ Rewrite the following loop as a list comprehension:

<div class="alert alert-block alert-info">
>>> sent = ['The', 'dog', 'gave', 'John', 'the', 'newspaper']<br>
>>> result = []<br>
>>> for word in sent:<br>
...     word_len = (word, len(word))<br>
...     result.append(word_len)<br>
>>> result<br>
[('The', 3), ('dog', 3), ('gave', 4), ('John', 4), ('the', 3), ('newspaper', 9)]
</div>    
    
○ Define a string raw containing a sentence of your own choosing. Now, split raw on some character other than space, such as 's'.

○ Write a for loop to print out the characters of a string, one per line.

○ What is the difference between calling split on a string with no argument and one with ' ' as the argument, e.g., sent.split() versus sent.split(' ')? What happens when the string being split contains tab characters, consecutive space characters, or a sequence of tabs and spaces? (In IDLE you will need to use '\t' to enter a tab character.)

○ Create a variable words containing a list of words. Experiment with words.sort() and sorted(words). What is the difference?

○ Explore the difference between strings and integers by typing the following at a Python prompt: "3" * 7 and 3 * 7. Try converting between strings and integers using int("3") and str(3).

○ Earlier, we asked you to use a text editor to create a file called test.py, containing the single line monty = 'Monty Python'. If you haven’t already done this (or can’t find the file), go ahead and do it now. Next, start up a new session with the Python interpreter, and enter the expression monty at the prompt. You will get an error from the interpreter. Now, try the following (note that you have to leave off the .py part of the filename):

<div class="alert alert-block alert-info">
>>> from test import msg<br>
>>> msg<br>
</div>    
    
This time, Python should return with a value. You can also try import test, in which case Python should be able to evaluate the expression test.monty at the prompt.

○ What happens when the formatting strings %6s and %-6s are used to display strings that are longer than six characters?

 Read in some text from a corpus, tokenize it, and print the list of all wh-word types that occur. (wh-words in English are used in questions, relative clauses, and exclamations: who, which, what, and so on.) Print them in order. Are any words duplicated in this list, because of the presence of case distinctions or punctuation?

 Create a file consisting of words and (made up) frequencies, where each line consists of a word, the space character, and a positive integer, e.g., fuzzy 53. Read the file into a Python list using open(filename).readlines(). Next, break each line into its two fields using split(), and convert the number into an integer using int(). The result should be a list of the form: [['fuzzy', 53], ...].

 Write code to access a favorite web page and extract some text from it. For example, access a weather site and extract the forecast top temperature for your town or city today.

 Write a function unknown() that takes a URL as its argument, and returns a list of unknown words that occur on that web page. In order to do this, extract all substrings consisting of lowercase letters (using re.findall()) and remove any items from this set that occur in the Words Corpus (nltk.corpus.words). Try to categorize these words manually and discuss your findings.

 Examine the results of processing the URL http://news.bbc.co.uk/ using the regular expressions suggested above. You will see that there is still a fair amount of non-textual data there, particularly JavaScript commands. You may also find that sentence breaks have not been properly preserved. Define further regular expressions that improve the extraction of text from this web page.

 Are you able to write a regular expression to tokenize text in such a way that the word don’t is tokenized into do and n’t? Explain why this regular expression won’t work: «n't|\w+».

 Try to write code to convert text into hAck3r, using regular expressions and substitution, where e → 3, i → 1, o → 0, l → |, s → 5, . → 5w33t!, ate → 8. Normalize the text to lowercase before converting it. Add more substitutions of your own. Now try to map s to two different values: $ for word-initial s, and 5 for word-internal s.

 Pig Latin is a simple transformation of English text. Each word of the text is converted as follows: move any consonant (or consonant cluster) that appears at the start of the word to the end, then append ay, e.g., string → ingstray, idle → idleay (see http://en.wikipedia.org/wiki/Pig_Latin).

Write a function to convert a word to Pig Latin.

Write code that converts text, instead of individual words.

Extend it further to preserve capitalization, to keep qu together (so that quiet becomes ietquay, for example), and to detect when y is used as a consonant (e.g., yellow) versus a vowel (e.g., style).

 Download some text from a language that has vowel harmony (e.g., Hungarian), extract the vowel sequences of words, and create a vowel bigram table.

 Python’s random module includes a function choice() which randomly chooses an item from a sequence; e.g., choice("aehh ") will produce one of four possible characters, with the letter h being twice as frequent as the others. Write a generator expression that produces a sequence of 500 randomly chosen letters drawn from the string "aehh ", and put this expression inside a call to the ''.join() function, to concatenate them into one long string. You should get a result that looks like uncontrolled sneezing or maniacal laughter: he haha ee heheeh eha. Use split() and join() again to normalize the whitespace in this string.

 Consider the numeric expressions in the following sentence from the MedLine Corpus: The corresponding free cortisol fractions in these sera were 4.53 +/- 0.15% and 8.16 +/- 0.23%, respectively. Should we say that the numeric expression 4.53 +/- 0.15% is three words? Or should we say that it’s a single compound word? Or should we say that it is actually nine words, since it’s read “four point five three, plus or minus fifteen percent”? Or should we say that it’s not a “real” word at all, since it wouldn’t appear in any dictionary? Discuss these different possibilities. Can you think of application domains that motivate at least two of these answers?

 Readability measures are used to score the reading difficulty of a text, for the purposes of selecting texts of appropriate difficulty for language learners. Let us define μw to be the average number of letters per word, and μs to be the average number of words per sentence, in a given text. The Automated Readability Index (ARI) of the text is defined to be: 4.71 μw + 0.5 μs - 21.43. Compute the ARI score for various sections of the Brown Corpus, including section f (popular lore) and j (learned). Make use of the fact that nltk.corpus.brown.words() produces a sequence of words, whereas nltk.corpus.brown.sents() produces a sequence of sentences.

 Use the Porter Stemmer to normalize some tokenized text, calling the stemmer on each word. Do the same thing with the Lancaster Stemmer, and see if you observe any differences.

 Define the variable saying to contain the list ['After', 'all', 'is', 'said', 'and', 'done', ',', 'more', 'is', 'said', 'than', 'done', '.']. Process the list using a for loop, and store the result in a new list lengths. Hint: begin by assigning the empty list to lengths, using lengths = []. Then each time through the loop, use append() to add another length value to the list.

 Define a variable silly to contain the string: 'newly formed bland ideas are inexpressible in an infuriating way'. (This happens to be the legitimate interpretation that bilingual English-Spanish speakers can assign to Chomsky’s famous nonsense phrase colorless green ideas sleep furiously, according to Wikipedia). Now write code to perform the following tasks:

Split silly into a list of strings, one per word, using Python’s split() operation, and save this to a variable called bland.

Extract the second letter of each word in silly and join them into a string, to get 'eoldrnnnna'.

Combine the words in bland back into a single string, using join(). Make sure the words in the resulting string are separated with whitespace.

Print the words of silly in alphabetical order, one per line.

 The index() function can be used to look up items in sequences. For example, 'inexpressible'.index('e') tells us the index of the first position of the letter e.

What happens when you look up a substring, e.g., 'inexpressible'.index('re')?

Define a variable words containing a list of words. Now use words.index() to look up the position of an individual word.

Define a variable silly as in Exercise 32. Use the index() function in combination with list slicing to build a list phrase consisting of all the words up to (but not including) in in silly.

 Write code to convert nationality adjectives such as Canadian and Australian to their corresponding nouns Canada and Australia (see http://en.wikipedia.org/wiki/List_of_adjectival_forms_of_place_names).

 Read the LanguageLog post on phrases of the form as best as p can and as best p can, where p is a pronoun. Investigate this phenomenon with the help of a corpus and the findall() method for searching tokenized text described in Useful Applications of Regular Expressions. The post is at http://itre.cis.upenn.edu/~myl/languagelog/archives/002733.html.

 Study the lolcat version of the book of Genesis, accessible as nltk.corpus.genesis.words('lolcat.txt'), and the rules for converting text into lolspeak at http://www.lolcatbible.com/index.php?title=How_to_speak_lolcat. Define regular expressions to convert English words into corresponding lolspeak words.

 Read about the re.sub() function for string substitution using regular expressions, using help(re.sub) and by consulting the further readings for this chapter. Use re.sub in writing code to remove HTML tags from an HTML file, and to normalize whitespace.

● An interesting challenge for tokenization is words that have been split across a linebreak. E.g., if long-term is split, then we have the string long-\nterm.

Write a regular expression that identifies words that are hyphenated at a line-break. The expression will need to include the \n character.

Use re.sub() to remove the \n character from these words.

How might you identify words that should not remain hyphenated once the newline is removed, e.g., 'encyclo-\npedia'?

● Read the Wikipedia entry on Soundex. Implement this algorithm in Python.

● Obtain raw texts from two or more genres and compute their respective reading difficulty scores as in the earlier exercise on reading difficulty. E.g., compare ABC Rural News and ABC Science News (nltk.corpus.abc). Use Punkt to perform sentence segmentation.

● Rewrite the following nested loop as a nested list comprehension:

<div class="alert alert-block alert-info">
>>> words = ['attribution', 'confabulation', 'elocution',<br>
...          'sequoia', 'tenacious', 'unidirectional']<br>
>>> vsequences = set()<br>
>>> for word in words:<br>
...     vowels = []<br>
...     for char in word:<br>
...         if char in 'aeiou':<br>
...             vowels.append(char)<br>
...     vsequences.add(''.join(vowels))<br>
>>> sorted(vsequences)<br>
['aiuio', 'eaiou', 'eouio', 'euoia', 'oauaio', 'uiieioa']
</div>

● Use WordNet to create a semantic index for a text collection. Extend the concordance search program in Example 3-1, indexing each word using the offset of its first synset, e.g., wn.synsets('dog')[0].offset (and optionally the offset of some of its ancestors in the hypernym hierarchy).

● With the help of a multilingual corpus such as the Universal Declaration of Human Rights Corpus (nltk.corpus.udhr), along with NLTK’s frequency distribution and rank correlation functionality (nltk.FreqDist, nltk.spearman_correlation), develop a system that guesses the language of a previously unseen text. For simplicity, work with a single character encoding and just a few languages.

● Write a program that processes a text and discovers cases where a word has been used with a novel sense. For each word, compute the WordNet similarity between all synsets of the word and all synsets of the words in its context. (Note that this is a crude approach; doing it well is a difficult, open research problem.)

● Read the article on normalization of non-standard words (Sproat et al., 2001), and implement a similar system for text normalization.